This is a scaled version of the model creation for the LSTM. In particular, since we'll be regularly creating a model for every stock we're interested in forecasting on a fairly regular basis, we've opted to only predict on one feature (the closing price), and to use an extremely limited LSTM (in terms of layers), with a limited number of epochs. In general, we also train this on a limited amount of data (e.g. since 2023) to improve performance. We've tested various iterations of time and number of features, and this seems to hit a good combination of performance and speed. 

Attached is a piece of code curating the particular number of stocks we're going to track. This can be scaled up. 

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import joblib

os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
import mlflow

yf.pdr_override()

# For time stamps
from datetime import datetime, timedelta


def makeModel(trainStock, trainStartDate):
    mlflow.autolog()
    df2 = pdr.get_data_yahoo(trainStock, start=trainStartDate, end=datetime.now())
    # Show the data
    # Create a new dataframe with only the 'Close column
    data = df2.filter(["Close"])
    # Convert the dataframe to a numpy array
    dataset = data.values
    # Get the number of rows to train the model on
    training_data_len = int(np.ceil(len(dataset) * 0.95))

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)

    train_data = scaled_data[0 : int(training_data_len), :]
    # Split the data into x_train and y_train data sets
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i - 60 : i, 0])
        y_train.append(train_data[i, 0])
        if i <= 61:
            print("61")

    # Convert the x_train and y_train to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compile the model

    # Train the model with mlflow
    mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
    mlflow.set_experiment(f"{trainStock}_{trainStartDate}_experiment")
    with mlflow.start_run() as run:
        model.fit(
            x_train,
            y_train,
            batch_size=1,
            epochs=2,
            callbacks=[mlflow.keras.MlflowCallback()],
        )

        # model.fit(x_train, y_train, batch_size=1, epochs=3)
        test_data = scaled_data[training_data_len - 60 :, :]
        # Create the data sets x_test and y_test
        x_test = []
        y_test = dataset[training_data_len:, :]
        for i in range(60, len(test_data)):
            x_test.append(test_data[i - 60 : i, 0])

        # Convert the data to a numpy array
        x_test = np.array(x_test)

        # Reshape the data
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

        # Get the models predicted price values
        predictions = model.predict(x_test)
        predictions = scaler.inverse_transform(predictions)

        # Get the root mean squared error (RMSE)
        rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
        mlflow.log_metric("rmse", rmse)

        model.save(
            rf"./models/{trainStock}.keras",
        )
        joblib.dump(
            scaler,
            rf"./models/{trainStock}.gz",
        )


In [ ]:
from lstm_model import makeModel

""" expanded_stock_list = [
    "A",
    "AA",
    "AAAP",
    "AABA",
    "AAC",
    "AAL",
    "AAMC",
    "AAME",
    "AAN",
    "AAOI",
    "AAON",
    "AAP",
    "AAPL",
    "AAT",
    "AAU",
    "AAV",
    "AAWW",
    "AAXN",
    "AB",
    "ABAC",
    "ABAX",
    "ABB",
    "ABBV",
    "ABC",
    "ABCB",
    "ABCD",
    "ABCO",
    "ABDC",
    "ABE",
    "ABEO",
    "ABEOW",
    "ABEV",
    "ABG",
    "ABIL",
    "ABIO",
    "ABLX",
    "ABM",
    "ABMD",
    "ABR",
    "ABRN",
    "ABR_A",
    "ABR_B",
    "ABR_C",
    "ABT",
    "ABTX",
    "ABUS",
    "ABX",
    "ABY",
    "AC",
    "ACAD",
    "ACBI",
    "ACC",
    "ACCO",
    "ACCP",
    "ACER",
    "ACERW",
    "ACET",
    "ACFC",
    "ACGL",
    "ACGLO",
    "ACGLP",
    "ACH",
    "ACHC",
    "ACHN",
    "ACHV",
    "ACIA",
    "ACIU",
    "ACIW",
    "ACLS",
    "ACM",
    "ACMR",
    "ACN",
    "ACNB",
    "ACOR",
    "ACP",
    "ACRE",
    "ACRS",
    "ACRX",
    "ACSF",
    "ACSI",
    "ACST",
    "ACTA",
    "ACTG",
    "ACU",
    "ACV",
    "ACXM",
    "ACY",
    "ADAP",
    "ADBE",
    "ADC",
    "ADES",
    "ADHD",
    "ADI",
    "ADM",
    "ADMA",
    "ADMP",
    "ADMS",
    "ADNT",
    "ADOM",
    "ADP",
    "ADRO",
    "ADS",
    "ADSK",
    "ADSW",
    "ADTN",
    "ADUS",
    "ADVM",
    "ADX",
    "ADXS",
    "ADXSW",
    "AE",
    "AEB",
    "AED",
    "AEE",
    "AEG",
    "AEGN",
    "AEH",
    "AEHR",
    "AEIS",
    "AEK",
    "AEL",
    "AEM",
    "AEMD",
    "AEO",
    "AEP",
    "AER",
    "AERI",
    "AES",
    "AET",
    "AETI",
    "AEUA",
    "AEY",
    "AEZS",
    "AFAM",
    "AFB",
    "AFC",
    "AFG",
    "AFGE",
    "AFGH",
    "AFH",
    "AFHBL",
    "AFI",
    "AFL",
    "AFMD",
    "AFSD",
    "AFSI",
    "AFSI_A",
    "AFSI_B",
    "AFSI_C",
    "AFSI_D",
    "AFSI_E",
    "AFSI_F",
    "AFSS",
    "AFST",
    "AFT",
    "AFTY",
    "AG",
    "AGC",
    "AGCO",
    "AGD",
    "AGEN",
    "AGFS",
    "AGFSW",
    "AGGE",
    "AGGP",
    "AGGY",
    "AGI",
    "AGII",
    "AGIIL",
    "AGIO",
    "AGLE",
    "AGM-A",
    "AGM",
    "AGM_A",
    "AGM_B",
    "AGM_C",
    "AGN",
    "AGNC",
    "AGNCB",
    "AGNCN",
    "AGN_A",
    "AGO",
    "AGO_B",
    "AGO_E",
    "AGO_F",
    "AGR",
    "AGRO",
    "AGRX",
    "AGT",
    "AGTC",
    "AGU",
    "AGX",
    "AGYS",
    "AHC",
    "AHGP",
    "AHH",
    "AHL",
    "AHL_C",
    "AHL_D",
    "AHP",
    "AHPA",
    "AHPAU",
    "AHPAW",
    "AHPI",
    "AHP_B",
    "AHT",
    "AHT_D",
    "AHT_F",
    "AHT_G",
    "AHT_H",
    "AI",
    "AIB-CL",
    "AIC",
    "AIEQ",
    "AIF",
    "AIG-WS",
    "AIG",
    "AIMC",
    "AIMT",
    "AIN",
    "AINC",
    "AINV",
    "AIR",
    "AIRG",
    "AIRI",
    "AIRT",
    "AIT",
    "AIV",
    "AIV_A",
    "AIW",
    "AIY",
    "AIZ",
    "AI_B",
    "AJG",
    "AJRD",
    "AJX",
    "AJXA",
    "AKAM",
    "AKAO",
    "AKBA",
    "AKCA",
    "AKER",
    "AKG",
    "AKO-A",
    "AKO-B",
    "AKP",
    "AKR",
    "AKRX",
    "AKS",
    "AKTS",
    "AKTX",
    "AL",
    "ALB",
    "ALBO",
    "ALCO",
    "ALDR",
    "ALDW",
    "ALDX",
    "ALE",
    "ALEX",
    "ALFI",
    "ALG",
    "ALGN",
    "ALGT",
    "ALIM",
    "ALJJ",
    "ALK",
    "ALKS",
    "ALL",
    "ALLE",
    "ALLT",
    "ALLY",
    "ALLY_A",
    "ALL_A",
    "ALL_B",
    "ALL_C",
    "ALL_D",
    "ALL_E",
    "ALL_F",
    "ALN",
    "ALNA",
    "ALNY",
    "ALO",
    "ALOG",
    "ALOT",
    "ALPN",
    "ALP_O-CL",
    "ALP_Q",
    "ALQA",
    "ALRM",
    "ALRN",
    "ALSK",
    "ALSN",
    "ALT",
    "ALTR",
    "ALTY",
    "ALV",
    "ALX",
    "ALXN",
    "AM",
    "AMAG",
    "AMAT",
    "AMBA",
    "AMBC",
    "AMBCW",
    "AMBR",
    "AMC",
    "AMCA",
    "AMCN",
    "AMCX",
    "AMD",
    "AMDA",
    "AME",
    "AMED",
    "AMFW",
    "AMG",
    "AMGN",
    "AMGP",
    "AMH",
    "AMH_C",
    "AMH_D",
    "AMH_E",
    "AMH_F",
    "AMH_G",
    "AMID",
    "AMKR",
    "AMLX",
    "AMMA",
    "AMN",
    "AMNB",
    "AMOT",
    "AMOV",
    "AMP",
    "AMPE",
    "AMPH",
    "AMRB",
    "AMRC",
    "AMRH",
    "AMRHW",
    "AMRK",
    "AMRN",
    "AMRS",
    "AMS",
    "AMSC",
    "AMSF",
    "AMSWA",
    "AMT",
    "AMTD",
    "AMTX",
    "AMT_B",
    "AMWD",
    "AMX",
    "AMZA",
    "AMZN",
    "AN",
    "ANAB",
    "ANAT",
    "ANCB",
    "ANCX",
    "ANDA",
    "ANDAR",
    "ANDAU",
    "ANDAW",
    "ANDE",
    "ANDV",
    "ANDX",
    "ANET",
    "ANF",
    "ANFI",
    "ANGI",
    "ANGO",
    "ANH",
    "ANH_A",
    "ANH_B",
    "ANH_C",
    "ANIK",
    "ANIP",
    "ANSS",
    "ANTH",
    "ANTM",
    "ANTX",
    "ANW",
    "ANY",
    "AOBC",
    "AOD",
    "AOI",
    "AON",
    "AOS",
    "AOSL",
    "AOXG",
    "AP",
    "APA",
    "APAM",
    "APB",
    "APC",
    "APD",
    "APDN",
    "APDNW",
    "APEI",
    "APEN",
    "APF",
    "APH",
    "APHB",
    "APLE",
    "APLP",
    "APLS",
    "APO",
    "APOG",
    "APOP",
    "APOPW",
    "APO_A",
    "APPF",
    "APPN",
    "APPS",
    "APRI",
    "APRN",
    "APT",
    "APTI",
    "APTO",
    "APTS",
    "APU",
    "APVO",
    "APWC",
    "AQ",
    "AQB",
    "AQMS",
    "AQN",
    "AQUA",
    "AQXP",
    "AR",
    "ARA",
    "ARAY",
    "ARC",
    "ARCB",
    "ARCC",
    "ARCH",
    "ARCI",
    "ARCM",
    "ARCO",
    "ARCW",
    "ARCX",
    "ARD",
    "ARDC",
    "ARDM",
    "ARDX",
    "ARE",
    "ARES",
    "ARES_A",
    "AREX",
    "ARE_D",
    "ARGS",
    "ARGX",
    "ARH_C",
    "ARI",
    "ARII",
    "ARI_C",
    "ARKG",
    "ARKK",
    "ARKQ",
    "ARKR",
    "ARKW",
    "ARL",
    "ARLP",
    "ARLZ",
    "ARMK",
    "ARNA",
    "ARNC",
    "ARNC_",
    "ARNC_B",
    "AROC",
    "AROW",
    "ARQL",
    "ARR",
    "ARRS",
    "ARRY",
    "ARR_A",
    "ARR_B",
    "ARTNA",
    "ARTW",
    "ARTX",
    "ARW",
    "ARWR",
    "ASA",
    "ASB-WS",
    "ASB",
    "ASB_C",
    "ASB_D",
    "ASC",
    "ASCMA",
    "ASET",
    "ASFI",
    "ASG",
    "ASGN",
    "ASH",
    "ASHX",
    "ASIX",
    "ASM",
    "ASMB",
    "ASML",
    "ASNA",
    "ASND",
    "ASNS",
    "ASPN",
    "ASPS",
    "ASPU",
    "ASR",
    "ASRV",
    "ASRVP",
    "AST",
    "ASTC",
    "ASTE",
    "ASUR",
    "ASV",
    "ASX",
    "ASYS",
    "AT",
    "ATAC",
    "ATACR",
    "ATACU",
    "ATAI",
    "ATAX",
    "ATEC",
    "ATEN",
    "ATGE",
    "ATH",
    "ATHM",
    "ATHN",
    "ATHX",
    "ATI",
    "ATKR",
    "ATLC",
    "ATLO",
    "ATNI",
    "ATNM",
    "ATNX",
    "ATO",
    "ATOM",
    "ATOS",
    "ATR",
    "ATRA",
    "ATRC",
    "ATRI",
    "ATRO",
    "ATRS",
    "ATSG",
    "ATTO",
    "ATTU",
    "ATU",
    "ATUS",
    "ATV",
    "ATVI",
    "ATXI",
    "AU",
    "AUBN",
    "AUDC",
    "AUG",
    "AUMN",
    "AUO",
    "AUPH",
    "AUTO",
    "AUY",
    "AVA",
    "AVAL",
    "AVAV",
    "AVB",
    "AVD",
    "AVDL",
    "AVEO",
    "AVGO",
    "AVGR",
    "AVH",
    "AVHI",
    "AVID",
    "AVIR",
    "AVK",
    "AVNW",
    "AVP",
    "AVT",
    "AVX",
    "AVXL",
    "AVXS",
    "AVY",
    "AWF",
    "AWI",
    "AWK",
    "AWP",
    "AWR",
    "AWRE",
    "AWX",
    "AXAS",
    "AXDX",
    "AXE",
    "AXGN",
    "AXL",
    "AXON",
    "AXP",
    "AXR",
    "AXS",
    "AXSM",
    "AXS_D",
    "AXS_E",
    "AXTA",
    "AXTI",
    "AXU",
    "AYI",
    "AYR",
    "AYTU",
    "AYX",
    "AZN",
    "AZO",
    "AZPN",
    "AZRE",
    "AZRX",
    "AZUL",
    "AZZ",
    "B",
    "BA",
    "BAA",
    "BABA",
    "BABY",
    "BAC-WS-A",
    "BAC-WS-B",
    "BAC",
    "BAC_A",
    "BAC_C",
    "BAC_D",
    "BAC_E",
    "BAC_I",
    "BAC_L",
    "BAC_W",
    "BAC_Y",
    "BAF",
    "BAH",
    "BAK",
    "BAM",
    "BANC",
    "BANC_C",
    "BANC_D",
    "BANC_E",
    "BAND",
    "BANF",
    "BANFP",
    "BANR",
    "BANX",
    "BAP",
    "BAR",
    "BAS",
    "BASI",
    "BATRA",
    "BATRK",
    "BAX",
    "BB",
    "BBBY",
    "BBC",
    "BBD",
    "BBDO",
    "BBF",
    "BBG",
    "BBGI",
    "BBK",
    "BBL",
    "BBN",
    "BBOX",
    "BBRG",
    "BBRX",
    "BBSI",
    "BBT",
    "BBT_D",
    "BBT_E",
    "BBT_F",
    "BBT_G",
    "BBT_H",
    "BBU",
    "BBVA",
    "BBW",
    "BBX",
    "BBY",
    "BC",
    "BCAC",
    "BCACR",
    "BCACU",
    "BCACW",
    "BCBP",
    "BCC",
    "BCD",
    "BCE",
    "BCEI",
    "BCH",
    "BCI",
    "BCLI",
    "BCO",
    "BCOM",
    "BCOR",
    "BCOV",
    "BCPC",
    "BCR",
    "BCRH",
    "BCRX",
    "BCS",
    "BCS_D",
    "BCTF",
    "BCV",
    "BCV_A",
    "BCX",
    "BDC",
    "BDCZ",
    "BDC_B",
    "BDGE",
    "BDJ",
    "BDL",
    "BDN",
    "BDR",
    "BDSI",
    "BDX",
    "BDXA",
    "BEAT",
    "BEBE",
    "BECN",
    "BEDU",
    "BEF",
    "BEL",
    "BELFA",
    "BELFB",
    "BEMO",
    "BEN",
    "BEP",
    "BERN",
    "BERY",
    "BETR",
    "BF-A",
    "BF-B",
    "BFAM",
    "BFIN",
    "BFIT",
    "BFK",
    "BFO",
    "BFR",
    "BFS",
    "BFS_C",
    "BFY",
    "BFZ",
    "BG",
    "BGB",
    "BGC",
    "BGCA",
    "BGCP",
    "BGFV",
    "BGG",
    "BGH",
    "BGI",
    "BGIO",
    "BGNE",
    "BGR",
    "BGS",
    "BGSF",
    "BGT",
    "BGX",
    "BGY",
    "BH",
    "BHAC",
    "BHACR",
    "BHACU",
    "BHACW",
    "BHB",
    "BHBK",
    "BHE",
    "BHF",
    "BHGE",
    "BHK",
    "BHLB",
    "BHP",
    "BHV",
    "BHVN",
    "BIBL",
    "BID",
    "BIDU",
    "BIF",
    "BIG",
    "BIIB",
    "BIO-B",
    "BIO",
    "BIOA",
    "BIOC",
    "BIOL",
    "BIOS",
    "BIP",
    "BIT",
    "BITA",
    "BIVV",
    "BJRI",
    "BJZ",
    "BK",
    "BKCC",
    "BKD",
    "BKE",
    "BKEP",
    "BKEPP",
    "BKH",
    "BKHU",
    "BKI",
    "BKJ",
    "BKK",
    "BKMU",
    "BKN",
    "BKS",
    "BKSC",
    "BKT",
    "BKU",
    "BKYI",
    "BK_C",
    "BL",
    "BLBD",
    "BLCM",
    "BLD",
    "BLDP",
    "BLDR",
    "BLE",
    "BLES",
    "BLFS",
    "BLH",
    "BLHY",
    "BLIN",
    "BLJ",
    "BLK",
    "BLKB",
    "BLL",
    "BLMN",
    "BLMT",
    "BLPH",
    "BLRX",
    "BLUE",
    "BLVD",
    "BLVDU",
    "BLVDW",
    "BLW",
    "BLX",
    "BMA",
    "BMCH",
    "BME",
    "BMI",
    "BMLA",
    "BMLP",
    "BML_G",
    "BML_H",
    "BML_I",
    "BML_J",
    "BML_L",
    "BMO",
    "BMRA",
    "BMRC",
    "BMRN",
    "BMS",
    "BMTC",
    "BMY",
    "BNCL",
    "BNDC",
    "BNED",
    "BNFT",
    "BNJ",
    "BNS",
    "BNSO",
    "BNTC",
    "BNTCW",
    "BNY",
    "BOBE",
    "BOCH",
    "BOE",
    "BOFI",
    "BOFIL",
    "BOH",
    "BOJA",
    "BOKF",
    "BOKFL",
    "BOLD",
    "BOLT",
    "BOMN",
    "BONT",
    "BOOM",
    "BOOT",
    "BORN",
    "BOSC",
    "BOSS",
    "BOTJ",
    "BOTZ",
    "BOX",
    "BOXL",
    "BP",
    "BPFH",
    "BPFHP",
    "BPFHW",
    "BPI",
    "BPK",
    "BPL",
    "BPMC",
    "BPMP",
    "BPMX",
    "BPOP",
    "BPOPM",
    "BPOPN",
    "BPRN",
    "BPT",
    "BPTH",
    "BPY",
    "BQH",
    "BR",
    "BRAC",
    "BRACR",
    "BRACU",
    "BRACW",
    "BRC",
    "BRCD",
    "BREW",
    "BRFS",
    "BRG",
    "BRGL",
    "BRG_A",
    "BRG_C",
    "BRG_D",
    "BRID",
    "BRK-A",
    "BRK-B",
    "BRKL",
    "BRKR",
    "BRKS",
    "BRN",
    "BRO",
    "BRQS",
    "BRQSW",
    "BRS",
    "BRSS",
    "BRT",
    "BRX",
    "BSAC",
    "BSBR",
    "BSCP",
    "BSCQ",
    "BSCR",
    "BSD",
    "BSE",
    "BSET",
    "BSF",
    "BSFT",
    "BSJN",
    "BSJO",
    "BSJP",
    "BSL",
    "BSM",
    "BSMX",
    "BSPM",
    "BSQR",
    "BSRR",
    "BST",
    "BSTC",
    "BSTI",
    "BSWN",
    "BSX",
    "BT",
    "BTA",
    "BTE",
    "BTEC",
    "BTG",
    "BTI",
    "BTN",
    "BTO",
    "BTT",
    "BTU",
    "BTU_",
    "BTX-WS",
    "BTX",
    "BTZ",
    "BUD",
    "BUFF",
    "BUI",
    "BUR",
    "BURL",
    "BUSE",
    "BUZ",
    "BV",
    "BVAL",
    "BVN",
    "BVSN",
    "BVX",
    "BVXV",
    "BVXVW",
    "BW",
    "BWA",
    "BWEN",
    "BWFG",
    "BWG",
    "BWINA",
    "BWINB",
    "BWL-A",
    "BWLD",
    "BWP",
    "BWXT",
    "BX",
    "BXC",
    "BXE",
    "BXG",
    "BXMT",
    "BXMX",
    "BXP",
    "BXP_B",
    "BXS",
    "BY",
    "BYBK",
    "BYD",
    "BYFC",
    "BYM",
    "BYSI",
    "BZH",
    "BZM",
    "BZUN",
    "C-WS-A",
    "C",
    "CA",
    "CAA",
    "CAAS",
    "CABO",
    "CAC",
    "CACC",
    "CACG",
    "CACI",
    "CADC",
    "CADE",
    "CAE",
    "CAF",
    "CAFD",
    "CAG",
    "CAH",
    "CAI",
    "CAJ",
    "CAKE",
    "CAL",
    "CALA",
    "CALD",
    "CALF",
    "CALI",
    "CALL",
    "CALM",
    "CALX",
    "CAMP",
    "CAMT",
    "CANF",
    "CAPL",
    "CAPR",
    "CAR",
    "CARA",
    "CARB",
    "CARG",
    "CARO",
    "CARS",
    "CART",
    "CARV",
    "CASC",
    "CASH",
    "CASI",
    "CASM",
    "CASS",
    "CASY",
    "CAT",
    "CATB",
    "CATC",
    "CATH",
    "CATM",
    "CATO",
    "CATS",
    "CATY",
    "CATYW",
    "CAVM",
    "CAW",
    "CB",
    "CBA",
    "CBAK",
    "CBAN",
    "CBAY",
    "CBB",
    "CBB_B",
    "CBD",
    "CBF",
    "CBFV",
    "CBG",
    "CBH",
    "CBI",
    "CBIO",
    "CBK",
    "CBL",
    "CBLI",
    "CBL_D",
    "CBL_E",
    "CBM",
    "CBMG",
    "CBMX",
    "CBMXW",
    "CBO",
    "CBOE",
    "CBPO",
    "CBPX",
    "CBRL",
    "CBS-A",
    "CBS",
    "CBSH",
    "CBSHP",
    "CBT",
    "CBTX",
    "CBU",
    "CBX",
    "CBZ",
    "CC",
    "CCA",
    "CCBG",
    "CCC",
    "CCCL",
    "CCCR",
    "CCD",
    "CCE",
    "CCF",
    "CCI",
    "CCIH",
    "CCI_A",
    "CCJ",
    "CCK",
    "CCL",
    "CCLP",
    "CCM",
    "CCMP",
    "CCNE",
    "CCO",
    "CCOI",
    "CCOR",
    "CCRC",
    "CCRN",
    "CCS",
    "CCU",
    "CCUR",
    "CCV-CL",
    "CCV",
    "CCXI",
    "CCZ",
    "CDC",
    "CDE",
    "CDEV",
    "CDK",
    "CDL",
    "CDNA",
    "CDNS",
    "CDOR",
    "CDR",
    "CDR_B",
    "CDR_C",
    "CDTI",
    "CDTX",
    "CDW",
    "CDXC",
    "CDXS",
    "CDZI",
    "CE",
    "CEA",
    "CECE",
    "CECO",
    "CEE",
    "CEF",
    "CEFS",
    "CEI",
    "CEL",
    "CELC",
    "CELG",
    "CELGZ",
    "CELH",
    "CELP",
    "CEM",
    "CEMB",
    "CEMI",
    "CEN",
    "CENT",
    "CENTA",
    "CENX",
    "CEO",
    "CEQP",
    "CERC",
    "CERCW",
    "CERN",
    "CERS",
    "CET",
    "CETV",
    "CETX",
    "CETXP",
    "CETXW",
    "CEV",
    "CEVA",
    "CEY",
    "CEZ",
    "CF",
    "CFA",
    "CFBI",
    "CFBK",
    "CFCO",
    "CFCOU",
    "CFCOW",
    "CFC_B",
    "CFFI",
    "CFFN",
    "CFG",
    "CFMS",
    "CFNB",
    "CFO",
    "CFR",
    "CFRX",
    "CFR_A",
    "CFX",
    "CG",
    "CGA",
    "CGBD",
    "CGEN",
    "CGG",
    "CGI",
    "CGIX",
    "CGNT",
    "CGNX",
    "CGO",
    "CH",
    "CHA",
    "CHAD",
    "CHCI",
    "CHCO",
    "CHCT",
    "CHD",
    "CHDN",
    "CHE",
    "CHEF",
    "CHEK",
    "CHEKW",
    "CHFC",
    "CHFN",
    "CHFS",
    "CHGG",
    "CHGX",
    "CHH",
    "CHI",
    "CHK",
    "CHKE",
    "CHKP",
    "CHKR",
    "CHK_D",
    "CHL",
    "CHMA",
    "CHMG",
    "CHMI",
    "CHMI_A",
    "CHN",
    "CHNR",
    "CHRS",
    "CHRW",
    "CHS",
    "CHSCL",
    "CHSCM",
    "CHSCN",
    "CHSCO",
    "CHSCP",
    "CHSP",
    "CHT",
    "CHTR",
    "CHU",
    "CHUBA",
    "CHUBK",
    "CHUY",
    "CHW",
    "CHY",
    "CI",
    "CIA",
    "CIB",
    "CIBR",
    "CIC-U",
    "CIC-WS",
    "CIC",
    "CID",
    "CIDM",
    "CIE",
    "CIEN",
    "CIF",
    "CIFS",
    "CIG-C",
    "CIG",
    "CIGI",
    "CII",
    "CIK",
    "CIL",
    "CIM",
    "CIM_A",
    "CIM_B",
    "CINF",
    "CINR",
    "CIO",
    "CIO_A",
    "CIR",
    "CISN-WS",
    "CISN",
    "CIT",
    "CIVB",
    "CIVBP",
    "CIVI",
    "CIX",
    "CIZ",
    "CIZN",
    "CJ",
    "CJJD",
    "CKH",
    "CKPT",
    "CKX",
    "CL",
    "CLAR",
    "CLB",
    "CLBS",
    "CLCT",
    "CLD",
    "CLDC",
    "CLDR",
    "CLDT",
    "CLDX",
    "CLF",
    "CLFD",
    "CLGX",
    "CLH",
    "CLI",
    "CLIR",
    "CLIRW",
    "CLLS",
    "CLM",
    "CLMT",
    "CLNE",
    "CLNS",
    "CLNS_B",
    "CLNS_C-CL",
    "CLNS_D",
    "CLNS_E",
    "CLNS_G",
    "CLNS_H",
    "CLNS_I",
    "CLNS_J",
    "CLNT",
    "CLPR",
    "CLR",
    "CLRB",
    "CLRBW",
    "CLRBZ",
    "CLRO",
    "CLS",
    "CLSD",
    "CLSN",
    "CLTL",
    "CLUB",
    "CLVS",
    "CLW",
    "CLWT",
    "CLX",
    "CLXT",
    "CLYH",
    "CM",
    "CMA-WS",
    "CMA",
    "CMC",
    "CMCL",
    "CMCM",
    "CMCO",
    "CMCSA",
    "CMCT",
    "CMD",
    "CME",
    "CMFN",
    "CMG",
    "CMI",
    "CMLS",
    "CMO",
    "CMO_E",
    "CMP",
    "CMPR",
    "CMRE",
    "CMRE_B",
    "CMRE_C",
    "CMRE_D",
    "CMRX",
    "CMS",
    "CMSS",
    "CMSSR",
    "CMSSU",
    "CMSSW",
    "CMS_B",
    "CMT",
    "CMTA",
    "CMTL",
    "CMU",
    "CNA",
    "CNAC",
    "CNACR",
    "CNACU",
    "CNACW",
    "CNAT",
    "CNBKA",
    "CNC",
    "CNCE",
    "CNCR",
    "CNDF",
    "CNDT",
    "CNET",
    "CNFR",
    "CNHI",
    "CNHX",
    "CNI",
    "CNIT",
    "CNK",
    "CNMD",
    "CNNX",
    "CNO",
    "CNOB",
    "CNP",
    "CNQ",
    "CNS",
    "CNSF",
    "CNSL",
    "CNTF",
    "CNTY",
    "CNX",
    "CNXC",
    "CNXN",
    "CNXR",
    "CNYA",
    "CO",
    "COBZ",
    "CODA",
    "CODI",
    "CODI_A",
    "CODX",
    "COE",
    "COF-WS",
    "COF",
    "COF_C",
    "COF_D",
    "COF_F",
    "COF_G",
    "COF_H",
    "COF_P",
    "COG",
    "COGT",
    "COHN",
    "COHR",
    "COHU",
    "COKE",
    "COL",
    "COLB",
    "COLL",
    "COLM",
    "COM",
    "COMB",
    "COMG",
    "COMM",
    "CONE",
    "CONN",
    "COO",
    "COOL",
    "COP",
    "COR",
    "CORE",
    "CORI",
    "CORR",
    "CORR_A",
    "CORT",
    "COR_A-CL",
    "COR_A",
    "COST",
    "COT",
    "COTV",
    "COTY",
    "COUP",
    "COWN",
    "COWNL",
    "COWZ",
    "CP",
    "CPA",
    "CPAC",
    "CPAH",
    "CPB",
    "CPE",
    "CPE_A",
    "CPF",
    "CPG",
    "CPHC",
    "CPHI",
    "CPIX",
    "CPK",
    "CPL",
    "CPLA",
    "CPLP",
    "CPN",
    "CPRT",
    "CPRX",
    "CPS",
    "CPSH",
    "CPSI",
    "CPSS",
    "CPST",
    "CPT",
    "CPTA",
    "CPTAG",
    "CPTAL",
    "CQH",
    "CQP",
    "CR",
    "CRAI",
    "CRAK",
    "CRAY",
    "CRBP",
    "CRC",
    "CRCM",
    "CRD-A",
    "CRD-B",
    "CREE",
    "CREG",
    "CRESY",
    "CRF",
    "CRH",
    "CRHM",
    "CRI",
    "CRIS",
    "CRK",
    "CRL",
    "CRM",
    "CRMD",
    "CRME",
    "CRMT",
    "CRNT",
    "CROX",
    "CRR",
    "CRS",
    "CRSP",
    "CRT",
    "CRTN",
    "CRTO",
    "CRUS",
    "CRVL",
    "CRVP",
    "CRVS",
    "CRWS",
    "CRY",
    "CRZO",
    "CS",
    "CSA",
    "CSB",
    "CSBK",
    "CSBR",
    "CSCO",
    "CSF",
    "CSFL",
    "CSGP",
    "CSGS",
    "CSII",
    "CSIQ",
    "CSL",
    "CSLT",
    "CSML",
    "CSOD",
    "CSPI",
    "CSQ",
    "CSRA",
    "CSS",
    "CSSE",
    "CSTE",
    "CSTM",
    "CSTR",
    "CSU",
    "CSV",
    "CSWC",
    "CSWI",
    "CSX",
    "CTAA",
    "CTAS",
    "CTB",
    "CTBB",
    "CTBI",
    "CTDD",
    "CTEK",
    "CTG",
    "CTHR",
    "CTIB",
    "CTIC",
    "CTL",
    "CTLT",
    "CTMX",
    "CTO",
    "CTR",
    "CTRE",
    "CTRL",
    "CTRN",
    "CTRP",
    "CTRV",
    "CTS",
    "CTSH",
    "CTSO",
    "CTT",
    "CTU",
    "CTV",
    "CTW",
    "CTWS",
    "CTX",
    "CTXR",
    "CTXRW",
    "CTXS",
    "CTY",
    "CTZ",
    "CUB",
    "CUBA",
    "CUBE",
    "CUBI",
    "CUBI_C",
    "CUBI_D",
    "CUBI_E",
    "CUBI_F",
    "CUBN",
    "CUBS",
    "CUDA",
    "CUI",
    "CUK",
    "CULP",
    "CUMB",
    "CUNB",
    "CUO",
    "CUR",
    "CUTR",
    "CUZ",
    "CVA",
    "CVBF",
    "CVCO",
    "CVCY",
    "CVE",
    "CVEO",
    "CVG",
    "CVGI",
    "CVGW",
    "CVI",
    "CVLT",
    "CVLY",
    "CVM-WS",
    "CVM",
    "CVNA",
    "CVO",
    "CVR",
    "CVRR",
    "CVRS",
    "CVS",
    "CVTI",
    "CVU",
    "CVV",
    "CVX",
    "CW",
    "CWAI",
    "CWAY",
    "CWBC",
    "CWCO",
    "CWEB",
    "CWH",
    "CWS",
    "CWST",
    "CWT",
    "CX",
    "CXDC",
    "CXE",
    "CXH",
    "CXO",
    "CXP",
    "CXRX",
    "CXSE",
    "CXW",
    "CY",
    "CYAD",
    "CYAN",
    "CYBE",
    "CYBR",
    "CYCC",
    "CYCCP",
    "CYD",
    "CYH",
    "CYHHZ",
    "CYOU",
    "CYRN",
    "CYRX",
    "CYRXW",
    "CYS",
    "CYS_A",
    "CYS_B",
    "CYTK",
    "CYTR",
    "CYTX",
    "CYTXW",
    "CZFC",
    "CZNC",
    "CZR",
    "CZWI",
    "CZZ",
    "C_C",
    "C_J",
    "C_K",
    "C_L",
    "C_N",
    "C_P",
    "C_S",
    "D",
    "DAC",
    "DAIO",
    "DAKT",
    "DAL",
    "DALT",
    "DAN",
    "DAR",
    "DARE",
    "DATA",
    "DAVE",
    "DAX",
    "DB",
    "DBD",
    "DBES",
    "DBIT",
    "DBL",
    "DBRT",
    "DBVT",
    "DCF",
    "DCI",
    "DCIX",
    "DCM",
    "DCO",
    "DCOM",
    "DCP",
    "DCPH",
    "DCT",
    "DCUD",
    "DD",
    "DDBI",
    "DDC",
    "DDD",
    "DDE",
    "DDEZ",
    "DDF",
    "DDJP",
    "DDLS",
    "DDR",
    "DDR_A",
    "DDR_J",
    "DDR_K",
    "DDS",
    "DDT",
    "DDWM",
    "DD_A",
    "DD_B",
    "DE",
    "DEA",
    "DECK",
    "DEEF",
    "DEFA",
    "DEI",
    "DEL",
    "DELT",
    "DELTW",
    "DEMG",
    "DEMS",
    "DENN",
    "DEO",
    "DEPO",
    "DERM",
    "DESC",
    "DESP",
    "DEST",
    "DEUS",
    "DEWJ",
    "DEX",
    "DEZU",
    "DF",
    "DFBG",
    "DFEN",
    "DFFN",
    "DFIN",
    "DFND",
    "DFNL",
    "DFP",
    "DFRG",
    "DFS",
    "DFS_B-CL",
    "DFS_B",
    "DFVL",
    "DFVS",
    "DG",
    "DGICA",
    "DGICB",
    "DGII",
    "DGLT",
    "DGLY",
    "DGSE",
    "DGX",
    "DHDG",
    "DHF",
    "DHG",
    "DHI",
    "DHIL",
    "DHR",
    "DHT",
    "DHVW",
    "DHX",
    "DHXM",
    "DHY",
    "DIAL",
    "DIAX",
    "DIN",
    "DIOD",
    "DIS",
    "DISCA",
    "DISCB",
    "DISCK",
    "DISH",
    "DIT",
    "DIVA",
    "DIVB",
    "DIVO",
    "DIVY",
    "DJCO",
    "DJD",
    "DK",
    "DKL",
    "DKS",
    "DKT",
    "DL",
    "DLA",
    "DLB",
    "DLBL",
    "DLBR",
    "DLBS",
    "DLHC",
    "DLNG",
    "DLNG_A",
    "DLPH",
    "DLR",
    "DLR_C",
    "DLR_G",
    "DLR_H",
    "DLR_I",
    "DLR_J",
    "DLTH",
    "DLTR",
    "DLX",
    "DM",
    "DMB",
    "DMF",
    "DMLP",
    "DMO",
    "DMPI",
    "DMRC",
    "DMRI",
    "DMRL",
    "DMTX",
    "DNB",
    "DNBF",
    "DNI",
    "DNKN",
    "DNN",
    "DNOW",
    "DNP",
    "DNR",
    "DO",
    "DOC",
    "DOOR",
    "DORM",
    "DOTA",
    "DOTAR",
    "DOTAU",
    "DOTAW",
    "DOV",
    "DOVA",
    "DOX",
    "DPG",
    "DPLO",
    "DPS",
    "DPST",
    "DPW",
    "DPZ",
    "DQ",
    "DRAD",
    "DRD",
    "DRE",
    "DRH",
    "DRI",
    "DRIO",
    "DRIOW",
    "DRNA",
    "DRQ",
    "DRRX",
    "DRUA",
    "DRYS",
    "DS",
    "DSE",
    "DSGX",
    "DSKE",
    "DSKEW",
    "DSL",
    "DSM",
    "DSPG",
    "DSS",
    "DST",
    "DSU",
    "DSW",
    "DSWL",
    "DSX",
    "DSXN",
    "DSX_B",
    "DS_B",
    "DS_C",
    "DS_D",
    "DTE",
    "DTEA",
    "DTF",
    "DTJ",
    "DTK",
    "DTLA_",
    "DTQ",
    "DTRM",
    "DTUL",
    "DTUS",
    "DTV",
    "DTY",
    "DTYL",
    "DTYS",
    "DUC",
    "DUK",
    "DUKH",
    "DUSA",
    "DUSL",
    "DVA",
    "DVAX",
    "DVCR",
    "DVD",
    "DVEM",
    "DVMT",
    "DVN",
    "DVP",
    "DWAC",
    "DWAQ",
    "DWAS",
    "DWAT",
    "DWCH",
    "DWDP",
    "DWFI",
    "DWIN",
    "DWLD",
    "DWLV",
    "DWPP",
    "DWSN",
    "DWT",
    "DWTR",
    "DX",
    "DXB",
    "DXC",
    "DXCM",
    "DXLG",
    "DXPE",
    "DXR",
    "DXTR",
    "DXUS",
    "DXYN",
    "DX_A",
    "DX_B",
    "DY",
    "DYB",
    "DYLS",
    "DYN-WS-A",
    "DYN",
    "DYNC",
    "DYNT",
    "DYN_A",
    "DYSL",
    "DZSI",
    "E",
    "EA",
    "EAB",
    "EACQ",
    "EACQU",
    "EACQW",
    "EAD",
    "EAE",
    "EAGL",
    "EAGLU",
    "EAGLW",
    "EAI",
    "EARN",
    "EARS",
    "EAT",
    "EBAY",
    "EBAYL",
    "EBF",
    "EBIO",
    "EBIX",
    "EBMT",
    "EBR-B",
    "EBR",
    "EBS",
    "EBSB",
    "EBTC",
    "EC",
    "ECA",
    "ECC",
    "ECCA",
    "ECCB",
    "ECCY",
    "ECCZ",
    "ECF",
    "ECF_A",
    "ECHO",
    "ECL",
    "ECOL",
    "ECOM",
    "ECPG",
    "ECR",
    "ECT",
    "ECYT",
    "ED",
    "EDAP",
    "EDBI",
    "EDD",
    "EDF",
    "EDGE",
    "EDGW",
    "EDI",
    "EDIT",
    "EDN",
    "EDOM",
    "EDOW",
    "EDR",
    "EDU",
    "EDUC",
    "EE",
    "EEA",
    "EEFT",
    "EEI",
    "EEMO",
    "EEMX",
    "EEP",
    "EEQ",
    "EEX",
    "EFAS",
    "EFAX",
    "EFBI",
    "EFC",
    "EFF",
    "EFII",
    "EFL",
    "EFNL",
    "EFOI",
    "EFR",
    "EFSC",
    "EFT",
    "EFX",
    "EGAN",
    "EGBN",
    "EGF",
    "EGHT",
    "EGI",
    "EGIF",
    "EGL",
    "EGLE",
    "EGLT",
    "EGN",
    "EGO",
    "EGOV",
    "EGP",
    "EGRX",
    "EGY",
    "EHI",
    "EHIC",
    "EHR",
    "EHT",
    "EHTH",
    "EIA",
    "EIG",
    "EIGI",
    "EIGR",
    "EIM",
    "EIO",
    "EIP",
    "EIV",
    "EIX",
    "EKSO",
    "EL",
    "ELC",
    "ELEC",
    "ELECU",
    "ELECW",
    "ELF",
    "ELGX",
    "ELJ",
    "ELLI",
    "ELLO",
    "ELMD",
    "ELON",
    "ELP",
    "ELS",
    "ELSE",
    "ELTK",
    "ELU",
    "ELVT",
    "ELY",
    "EMAN",
    "EMBH",
    "EMBU",
    "EMCF",
    "EMCI",
    "EMD",
    "EMDV",
    "EME",
    "EMES",
    "EMF",
    "EMGF",
    "EMHY",
    "EMI",
    "EMITF",
    "EMJ",
    "EMKR",
    "EML",
    "EMMS",
    "EMN",
    "EMO",
    "EMP",
    "EMQQ",
    "EMR",
    "EMSD",
    "EMTL",
    "EMX",
    "EMXC",
    "ENB",
    "ENBL",
    "ENDP",
    "ENFC",
    "ENG",
    "ENH_C",
    "ENIA",
    "ENIC",
    "ENJ",
    "ENLC",
    "ENLK",
    "ENO",
    "ENOR",
    "ENPH",
    "ENR",
    "ENRJ",
    "ENS",
    "ENSG",
    "ENSV",
    "ENT",
    "ENTA",
    "ENTG",
    "ENTL",
    "ENTR",
    "ENV",
    "ENVA",
    "ENX",
    "ENZ",
    "ENZY",
    "EOCC",
    "EOD",
    "EOG",
    "EOI",
    "EOS",
    "EOT",
    "EPAM",
    "EPAY",
    "EPC",
    "EPD",
    "EPE",
    "EPIX",
    "EPM",
    "EPR",
    "EPRF",
    "EPR_C",
    "EPR_E",
    "EPR_F",
    "EPZM",
    "EP_C",
    "EQBK",
    "EQC",
    "EQCO",
    "EQC_D",
    "EQFN",
    "EQGP",
    "EQIX",
    "EQM",
    "EQR",
    "EQRR",
    "EQS",
    "EQT",
    "ERA",
    "ERC",
    "ERF",
    "ERGF",
    "ERH",
    "ERI",
    "ERIC",
    "ERIE",
    "ERII",
    "ERJ",
    "ERM",
    "ERN",
    "EROS",
    "ERYP",
    "ES",
    "ESBA",
    "ESBK",
    "ESCA",
    "ESDI",
    "ESDIW",
    "ESE",
    "ESEA",
    "ESES",
    "ESG",
    "ESGD",
    "ESGE",
    "ESGF",
    "ESGG",
    "ESGL",
    "ESGN",
    "ESGR",
    "ESGS",
    "ESGU",
    "ESGW",
    "ESIO",
    "ESL",
    "ESLT",
    "ESNC",
    "ESND",
    "ESNT",
    "ESP",
    "ESPR",
    "ESQ",
    "ESRT",
    "ESRX",
    "ESS",
    "ESSA",
    "ESTE",
    "ESV",
    "ESXB",
    "ETB",
    "ETE",
    "ETFC",
    "ETG",
    "ETH",
    "ETHO",
    "ETJ",
    "ETM",
    "ETMW",
    "ETN",
    "ETO",
    "ETP",
    "ETR",
    "ETSY",
    "ETV",
    "ETW",
    "ETX",
    "ETY",
    "EUDV",
    "EUFL",
    "EURN",
    "EURZ",
    "EUXL",
    "EV",
    "EVA",
    "EVAR",
    "EVBG",
    "EVBN",
    "EVC",
    "EVEP",
    "EVF",
    "EVG",
    "EVGBC",
    "EVGN",
    "EVH",
    "EVHC",
    "EVI",
    "EVIX",
    "EVJ",
    "EVK",
    "EVLMC",
    "EVLV",
    "EVM",
    "EVN",
    "EVO",
    "EVOK",
    "EVOL",
    "EVP",
    "EVR",
    "EVRI",
    "EVSTC",
    "EVT",
    "EVTC",
    "EVV",
    "EVY",
    "EW",
    "EWBC",
    "EWGS",
    "EWMC",
    "EWRE",
    "EWSC",
    "EWUS",
    "EXA",
    "EXAC",
    "EXAS",
    "EXC",
    "EXD",
    "EXEL",
    "EXFO",
    "EXG",
    "EXIV",
    "EXK",
    "EXLS",
    "EXP",
    "EXPD",
    "EXPE",
    "EXPO",
    "EXPR",
    "EXR",
    "EXTN",
    "EXTR",
    "EXXI",
    "EYE",
    "EYEG",
    "EYEGW",
    "EYES",
    "EYESW",
    "EYLD",
    "EZPW",
    "EZT",
    "F",
    "FAAR",
    "FAC",
    "FAF",
    "FALN",
    "FANG",
    "FANH",
    "FANZ",
    "FARM",
    "FARO",
    "FAST",
    "FAT",
    "FATE",
    "FAX",
    "FB",
    "FBC",
    "FBHS",
    "FBIO",
    "FBIZ",
    "FBK",
    "FBM",
    "FBMS",
    "FBNC",
    "FBNK",
    "FBP",
    "FBR",
    "FBSS",
    "FC",
    "FCAL",
    "FCAP",
    "FCAU",
    "FCB",
    "FCBC",
    "FCCO",
    "FCCY",
    "FCE-A",
    "FCEF",
    "FCEL",
    "FCF",
    "FCFS",
    "FCN",
    "FCNCA",
    "FCO",
    "FCPT",
    "FCRE",
    "FCSC",
    "FCVT",
    "FCX",
    "FDBC",
    "FDC",
    "FDEF",
    "FDEU",
    "FDLO",
    "FDMO",
    "FDP",
    "FDRR",
    "FDS",
    "FDUS",
    "FDVV",
    "FDX",
    "FE",
    "FEDU",
    "FEIM",
    "FELE",
    "FELP",
    "FEN",
    "FENC",
    "FENG",
    "FET",
    "FEYE",
    "FF",
    "FFBC",
    "FFBCW",
    "FFBW",
    "FFC",
    "FFG",
    "FFHG",
    "FFHL",
    "FFIC",
    "FFIN",
    "FFIU",
    "FFIV",
    "FFKT",
    "FFNW",
    "FFSG",
    "FFTG",
    "FFTI",
    "FFTY",
    "FFWM",
    "FGBI",
    "FGEN",
    "FGL",
    "FGP",
    "FH",
    "FHB",
    "FHN",
    "FHN_A",
    "FI",
    "FIBK",
    "FIBR",
    "FICO",
    "FIEE",
    "FIG",
    "FIHD",
    "FII",
    "FINL",
    "FINX",
    "FIS",
    "FISI",
    "FISK",
    "FISV",
    "FIT",
    "FITB",
    "FITBI",
    "FIV",
    "FIVE",
    "FIVN",
    "FIX",
    "FIXD",
    "FIZZ",
    "FL",
    "FLAT",
    "FLAU",
    "FLBR",
    "FLC",
    "FLCA",
    "FLCH",
    "FLCO",
    "FLDM",
    "FLEU",
    "FLEX",
    "FLGB",
    "FLGR",
    "FLGT",
    "FLHK",
    "FLIC",
    "FLIO",
    "FLIR",
    "FLJH",
    "FLJP",
    "FLKR",
    "FLKS",
    "FLL",
    "FLLV",
    "FLMB",
    "FLMX",
    "FLO",
    "FLOW",
    "FLQD",
    "FLQE",
    "FLQG",
    "FLQH",
    "FLQL",
    "FLQM",
    "FLQS",
    "FLR",
    "FLS",
    "FLT",
    "FLWS",
    "FLXN",
    "FLXS",
    "FLY",
    "FMAO",
    "FMAX",
    "FMBH",
    "FMBI",
    "FMC",
    "FMCI",
    "FMCIR",
    "FMCIU",
    "FMCIW",
    "FMDG",
    "FMHI",
    "FMI",
    "FMN",
    "FMNB",
    "FMO",
    "FMS",
    "FMSA",
    "FMX",
    "FN",
    "FNB",
    "FNBG",
    "FNB_E",
    "FNCF",
    "FND",
    "FNF",
    "FNFV",
    "FNG",
    "FNGN",
    "FNHC",
    "FNJN",
    "FNKO",
    "FNLC",
    "FNSR",
    "FNTE",
    "FNTEU",
    "FNTEW",
    "FNV",
    "FNWB",
    "FOANC",
    "FOE",
    "FOF",
    "FOGO",
    "FOLD",
    "FOMX",
    "FONR",
    "FOR",
    "FORD",
    "FORK",
    "FORM",
    "FORR",
    "FORTY",
    "FOSL",
    "FOX",
    "FOXA",
    "FOXF",
    "FPAY",
    "FPEI",
    "FPH",
    "FPI",
    "FPI_B",
    "FPP-WS",
    "FPP",
    "FPRX",
    "FPT",
    "FQAL",
    "FR",
    "FRA",
    "FRAC",
    "FRAN",
    "FRBA",
    "FRBK",
    "FRC",
    "FRC_C",
    "FRC_D",
    "FRC_E",
    "FRC_F",
    "FRC_G",
    "FRC_H",
    "FRD",
    "FRED",
    "FRGI",
    "FRME",
    "FRO",
    "FRPH",
    "FRPT",
    "FRSH",
    "FRSX",
    "FRT",
    "FRTA",
    "FRT_C",
    "FSAC",
    "FSACU",
    "FSACW",
    "FSAM",
    "FSB",
    "FSBC",
    "FSBK",
    "FSBW",
    "FSCT",
    "FSFG",
    "FSI",
    "FSIC",
    "FSLR",
    "FSM",
    "FSNN",
    "FSP",
    "FSS",
    "FSTR",
    "FSV",
    "FT",
    "FTAG",
    "FTAI",
    "FTD",
    "FTEK",
    "FTEO",
    "FTF",
    "FTFT",
    "FTI",
    "FTK",
    "FTNT",
    "FTR",
    "FTRI",
    "FTRPR",
    "FTS",
    "FTV",
    "FTVA",
    "FTXD",
    "FTXG",
    "FTXH",
    "FTXL",
    "FTXN",
    "FTXO",
    "FTXR",
    "FUL",
    "FULLL",
    "FULT",
    "FUN",
    "FUNC",
    "FUND",
    "FUSB",
    "FUT",
    "FUV",
    "FVAL",
    "FVC",
    "FVE",
    "FWONA",
    "FWONK",
    "FWP",
    "FWRD",
    "FXEP",
    "FXJP",
    "G",
    "GAB",
    "GABC",
    "GABR",
    "GABRW",
    "GAB_D",
    "GAB_G",
    "GAB_H",
    "GAB_J",
    "GAIA",
    "GAIN",
    "GAINM",
    "GAINN",
    "GAINO",
    "GALE",
    "GALT",
    "GAM",
    "GAMR",
    "GAM_B",
    "GARD",
    "GARS",
    "GASS",
    "GASX",
    "GATX",
    "GAZB",
    "GBAB",
    "GBCI",
    "GBDC",
    "GBIL",
    "GBL",
    "GBLI",
    "GBLIL",
    "GBLIZ",
    "GBNK",
    "GBR",
    "GBT",
    "GBX",
    "GCAP",
    "GCBC",
    "GCH",
    "GCI",
    "GCO",
    "GCOW",
    "GCP",
    "GCV",
    "GCVRZ",
    "GCV_B",
    "GD",
    "GDDY",
    "GDEN",
    "GDI",
    "GDL",
    "GDL_B",
    "GDO",
    "GDOT",
    "GDP",
    "GDS",
    "GDV",
    "GDVD",
    "GDV_A",
    "GDV_D",
    "GDV_G",
    "GE",
    "GEB",
    "GEC",
    "GECC",
    "GECCL",
    "GEF-B",
    "GEF",
    "GEH",
    "GEK",
    "GEL",
    "GEM",
    "GEMP",
    "GEN",
    "GENC",
    "GENE",
    "GENY",
    "GEO",
    "GEOS",
    "GER",
    "GERN",
    "GES",
    "GEVO",
    "GF",
    "GFA",
    "GFED",
    "GFF",
    "GFI",
    "GFN",
    "GFNCP",
    "GFNSL",
    "GFY",
    "GG",
    "GGAL",
    "GGB",
    "GGG",
    "GGM",
    "GGN",
    "GGN_B",
    "GGO",
    "GGO_A",
    "GGP",
    "GGP_A",
    "GGT",
    "GGT_B",
    "GGT_E",
    "GGZ",
    "GGZR",
    "GGZRW",
    "GGZ_A",
    "GHC",
    "GHDX",
    "GHL",
    "GHM",
    "GHS",
    "GHY",
    "GHYB",
    "GHYG",
    "GIB",
    "GIFI",
    "GIGA",
    "GIGB",
    "GIGM",
    "GIII",
    "GIL",
    "GILD",
    "GILT",
    "GIM",
    "GIMO",
    "GIS",
    "GJH",
    "GJO",
    "GJP",
    "GJR",
    "GJS",
    "GJT",
    "GJV",
    "GKOS",
    "GLAD",
    "GLADN",
    "GLBL",
    "GLBR",
    "GLBS",
    "GLBZ",
    "GLDD",
    "GLDW",
    "GLMD",
    "GLNG",
    "GLO",
    "GLOB",
    "GLOG",
    "GLOG_A",
    "GLOP",
    "GLOP_A",
    "GLOW",
    "GLP",
    "GLPG",
    "GLPI",
    "GLQ",
    "GLRE",
    "GLT",
    "GLU",
    "GLUU",
    "GLU_A",
    "GLV",
    "GLW",
    "GLYC",
    "GM-WS-B",
    "GM",
    "GME",
    "GMED",
    "GMFL",
    "GMLP",
    "GMLPP",
    "GMO",
    "GMRE",
    "GMRE_A",
    "GMS",
    "GMTA",
    "GMZ",
    "GNBC",
    "GNC",
    "GNCA",
    "GNCMA",
    "GNE",
    "GNE_A",
    "GNK",
    "GNL",
    "GNL_A",
    "GNMK",
    "GNMX",
    "GNRC",
    "GNRT",
    "GNRX",
    "GNST",
    "GNT",
    "GNTX",
    "GNTY",
    "GNT_A",
    "GNUS",
    "GNW",
    "GOAU",
    "GOEX",
    "GOF",
    "GOGL",
    "GOGO",
    "GOL",
    "GOLD",
    "GOLF",
    "GOOD",
    "GOODM",
    "GOODO",
    "GOODP",
    "GOOG",
    "GOOGL",
    "GOOS",
    "GOP",
    "GORO",
    "GOV",
    "GOVNI",
    "GPAC",
    "GPACU",
    "GPACW",
    "GPC",
    "GPE_A-CL",
    "GPI",
    "GPIC",
    "GPJA",
    "GPK",
    "GPL",
    "GPM",
    "GPMT",
    "GPMTW",
    "GPN",
    "GPOR",
    "GPP",
    "GPRE",
    "GPRK",
    "GPRO",
    "GPS",
    "GPT",
    "GPT_A",
    "GPX",
    "GRA",
    "GRAM",
    "GRBK",
    "GRC",
    "GRF",
    "GRFS",
    "GRIF",
    "GRMN",
    "GRMY",
    "GRNB",
    "GROW",
    "GRP-U",
    "GRPN",
    "GRR",
    "GRUB",
    "GRVY",
    "GRX",
    "GRX_A",
    "GRX_B",
    "GS",
    "GSAT",
    "GSB",
    "GSBC",
    "GSBD",
    "GSD",
    "GSEU",
    "GSEW",
    "GSH",
    "GSHT",
    "GSHTU",
    "GSHTW",
    "GSIE",
    "GSIT",
    "GSJY",
    "GSK",
    "GSL",
    "GSLC",
    "GSL_B",
    "GSM",
    "GSS",
    "GSSC",
    "GST",
    "GST_A",
    "GST_B",
    "GSUM",
    "GSV",
    "GSVC",
    "GS_A",
    "GS_B",
    "GS_C",
    "GS_D",
    "GS_I-CL",
    "GS_I",
    "GS_J",
    "GS_K",
    "GS_N",
    "GT",
    "GTE",
    "GTHX",
    "GTIM",
    "GTLS",
    "GTN-A",
    "GTN",
    "GTO",
    "GTS",
    "GTT",
    "GTXI",
    "GTY",
    "GTYH",
    "GTYHU",
    "GTYHW",
    "GUDB",
    "GURE",
    "GUT",
    "GUT_A",
    "GUT_C",
    "GV",
    "GVA",
    "GVIP",
    "GVP",
    "GWB",
    "GWGH",
    "GWPH",
    "GWR",
    "GWRE",
    "GWRS",
    "GWW",
    "GXP",
    "GYB",
    "GYC",
    "GYRO",
    "GZT",
    "H",
    "HA",
    "HABT",
    "HACK",
    "HACV",
    "HACW",
    "HAE",
    "HAFC",
    "HAHA",
    "HAIN",
    "HAIR",
    "HAL",
    "HALL",
    "HALO",
    "HAS",
    "HASI",
    "HAUD",
    "HAWK",
    "HAWX",
    "HAYN",
    "HAYU",
    "HBAN",
    "HBANN",
    "HBANO",
    "HBANP",
    "HBB",
    "HBCP",
    "HBHC",
    "HBHCL",
    "HBI",
    "HBIO",
    "HBK",
    "HBM-WS",
    "HBM",
    "HBMD",
    "HBNC",
    "HBP",
    "HCA",
    "HCAC-U",
    "HCAC-WS",
    "HCAC",
    "HCAP",
    "HCAPZ",
    "HCC",
    "HCCI",
    "HCHC",
    "HCI",
    "HCKT",
    "HCLP",
    "HCM",
    "HCN",
    "HCN_I",
    "HCOM",
    "HCP",
    "HCRF",
    "HCSG",
    "HD",
    "HDAW",
    "HDB",
    "HDEE",
    "HDEF",
    "HDEZ",
    "HDLV",
    "HDMV",
    "HDNG",
    "HDP",
    "HDRW",
    "HDS",
    "HDSN",
    "HE",
    "HEAR",
    "HEB",
    "HEBT",
    "HEES",
    "HEFV",
    "HEI-A",
    "HEI",
    "HELE",
    "HEMV",
    "HEP",
    "HEQ",
    "HES",
    "HESM",
    "HES_A",
    "HEUS",
    "HEUV",
    "HEWC",
    "HEWI",
    "HEWL",
    "HEWP",
    "HEWU",
    "HEWW",
    "HEWY",
    "HE_U",
    "HF",
    "HFBC",
    "HFBL",
    "HFC",
    "HFRO",
    "HFWA",
    "HFXE",
    "HFXI",
    "HFXJ",
    "HGH",
    "HGSD",
    "HGSH",
    "HGT",
    "HGV",
    "HHC",
    "HHS",
    "HHYX",
    "HI",
    "HIBB",
    "HIE",
    "HIFR",
    "HIFS",
    "HIG-WS",
    "HIG",
    "HIHO",
    "HII",
    "HIIQ",
    "HIL",
    "HIMX",
    "HIO",
    "HIPS",
    "HIVE",
    "HIW",
    "HIX",
    "HJPX",
    "HJV",
    "HK-WS",
    "HK",
    "HL",
    "HLF",
    "HLG",
    "HLI",
    "HLIT",
    "HLM_",
    "HLNE",
    "HLS",
    "HLT",
    "HLTH",
    "HLX",
    "HL_B",
    "HMC",
    "HMG",
    "HMHC",
    "HMLP",
    "HMLP_A",
    "HMN",
    "HMNF",
    "HMNY",
    "HMST",
    "HMSY",
    "HMTA",
    "HMTV",
    "HMY",
    "HNI",
    "HNNA",
    "HNP",
    "HNRG",
    "HNW",
    "HOFT",
    "HOG",
    "HOLI",
    "HOLX",
    "HOMB",
    "HOME",
    "HOML",
    "HON",
    "HONE",
    "HOPE",
    "HOS",
    "HOTR",
    "HOV",
    "HOVNP",
    "HP",
    "HPE",
    "HPF",
    "HPI",
    "HPJ",
    "HPP",
    "HPQ",
    "HPS",
    "HPT",
    "HQCL",
    "HQH",
    "HQL",
    "HQY",
    "HR",
    "HRB",
    "HRC",
    "HRG",
    "HRI",
    "HRL",
    "HRS",
    "HRTG",
    "HRTX",
    "HRZN",
    "HSBC",
    "HSBC_A",
    "HSC",
    "HSCZ",
    "HSEA",
    "HSEB",
    "HSGX",
    "HSIC",
    "HSII",
    "HSKA",
    "HSNI",
    "HSON",
    "HST",
    "HSTM",
    "HSY",
    "HT",
    "HTA",
    "HTBI",
    "HTBK",
    "HTBX",
    "HTD",
    "HTF-CL",
    "HTFA",
    "HTGC",
    "HTGM",
    "HTGX",
    "HTH",
    "HTHT",
    "HTLD",
    "HTLF",
    "HTM",
    "HTRB",
    "HTUS",
    "HTY",
    "HTZ",
    "HT_C",
    "HT_D",
    "HT_E",
    "HUBB",
    "HUBG",
    "HUBS",
    "HUM",
    "HUN",
    "HUNT",
    "HUNTU",
    "HUNTW",
    "HURC",
    "HURN",
    "HUSA",
    "HUSV",
    "HVBC",
    "HVT-A",
    "HVT",
    "HWBK",
    "HWCC",
    "HWKN",
    "HX",
    "HXL",
    "HY",
    "HYACU",
    "HYB",
    "HYDB",
    "HYDD",
    "HYGS",
    "HYH",
    "HYHG",
    "HYI",
    "HYLB",
    "HYLV",
    "HYT",
    "HYXE",
    "HYXU",
    "HZN",
    "HZNP",
    "HZO",
    "I",
    "IAC",
    "IAE",
    "IAF",
    "IAG",
    "IAGG",
    "IAM",
    "IAMXR",
    "IAMXW",
    "IART",
    "IBA",
    "IBCP",
    "IBD",
    "IBDR",
    "IBDS",
    "IBIO",
    "IBKC",
    "IBKCO",
    "IBKCP",
    "IBKR",
    "IBM",
    "IBMJ",
    "IBMK",
    "IBML",
    "IBN",
    "IBO",
    "IBOC",
    "IBP",
    "IBTX",
    "IBUY",
    "ICAD",
    "ICAN",
    "ICB",
    "ICBK",
    "ICCC",
    "ICCH",
    "ICD",
    "ICE",
    "ICFI",
    "ICHR",
    "ICL",
    "ICLR",
    "ICON",
    "ICOW",
    "ICPT",
    "ICSH",
    "ICUI",
    "ICVT",
    "IDA",
    "IDCC",
    "IDE",
    "IDEV",
    "IDHD",
    "IDLB",
    "IDMO",
    "IDN",
    "IDRA",
    "IDSA",
    "IDSY",
    "IDT",
    "IDTI",
    "IDXG",
    "IDXX",
    "IEC",
    "IEP",
    "IESC",
    "IEX",
    "IF",
    "IFF",
    "IFIX",
    "IFLY",
    "IFMK",
    "IFN",
    "IFON",
    "IFRX",
    "IGA",
    "IGC",
    "IGD",
    "IGEB",
    "IGEM",
    "IGHG",
    "IGI",
    "IGLD",
    "IGR",
    "IGRO",
    "IGT",
    "IGVT",
    "IGZ",
    "IHC",
    "IHD",
    "IHG",
    "IHIT",
    "IHT",
    "IID",
    "IIF",
    "III",
    "IIIN",
    "IIJI",
    "IIM",
    "IIN",
    "IIPR",
    "IIPR_A",
    "IIVI",
    "IKNX",
    "ILG",
    "ILMN",
    "IMAX",
    "IMDZ",
    "IMGN",
    "IMH",
    "IMI",
    "IMKTA",
    "IMMR",
    "IMMU",
    "IMMY",
    "IMNP",
    "IMO",
    "IMOM",
    "IMOS",
    "IMPV",
    "IMRN",
    "IMRNW",
    "IMTB",
    "IMTE",
    "IMUC-WS",
    "IMUC",
    "INAP",
    "INB",
    "INBK",
    "INBKL",
    "INCR",
    "INCY",
    "INDB",
    "INDF",
    "INDU",
    "INDUU",
    "INDUW",
    "INF",
    "INFI",
    "INFN",
    "INFO",
    "INFR",
    "INFU",
    "INFY",
    "ING",
    "INGN",
    "INGR",
    "INN",
    "INN_B-CL",
    "INN_B",
    "INN_C",
    "INN_D",
    "INO",
    "INOD",
    "INOV",
    "INPX",
    "INS",
    "INSE",
    "INSG",
    "INSI",
    "INSM",
    "INST",
    "INSW",
    "INSY",
    "INT",
    "INTC",
    "INTG",
    "INTL",
    "INTT",
    "INTU",
    "INTX",
    "INUV",
    "INVA",
    "INVE",
    "INVH",
    "INWK",
    "INXN",
    "IO",
    "IONS",
    "IOR",
    "IOSP",
    "IOTS",
    "IOVA",
    "IP",
    "IPAR",
    "IPAS",
    "IPAY",
    "IPB",
    "IPCC",
    "IPCI",
    "IPDN",
    "IPG",
    "IPGP",
    "IPHI",
    "IPHS",
    "IPI",
    "IPL_D",
    "IPOA-U",
    "IPOA-WS",
    "IPOA",
    "IPOS",
    "IPWR",
    "IPXL",
    "IQDG",
    "IQI",
    "IR",
    "IRBT",
    "IRCP",
    "IRDM",
    "IRDMB",
    "IRET",
    "IRET_B-CL",
    "IRET_C",
    "IRIX",
    "IRL",
    "IRLR",
    "IRM",
    "IRMD",
    "IROQ",
    "IRR",
    "IRS",
    "IRT",
    "IRTC",
    "IRWD",
    "ISBC",
    "ISCA",
    "ISD",
    "ISDR",
    "ISF",
    "ISG",
    "ISIG",
    "ISL",
    "ISM",
    "ISMD",
    "ISNS",
    "ISP-CL",
    "ISR",
    "ISRG",
    "ISRL",
    "ISSC",
    "ISTR",
    "ISZE",
    "IT",
    "ITCB",
    "ITCI",
    "ITEK",
    "ITEQ",
    "ITG",
    "ITGR",
    "ITI",
    "ITIC",
    "ITRI",
    "ITRN",
    "ITT",
    "ITUB",
    "ITUS",
    "ITW",
    "IVAC",
    "IVAL",
    "IVC",
    "IVENC",
    "IVFGC",
    "IVFVC",
    "IVH",
    "IVLU",
    "IVR",
    "IVR_A",
    "IVR_B",
    "IVR_C",
    "IVTY",
    "IVZ",
    "IX",
    "IXYS",
    "IYLD",
    "IZEA",
    "JACK",
    "JAG",
    "JAGX",
    "JAKK",
    "JASN",
    "JASNW",
    "JASO",
    "JAX",
    "JAZZ",
    "JBGS",
    "JBHT",
    "JBK",
    "JBL",
    "JBLU",
    "JBN",
    "JBR",
    "JBSS",
    "JBT",
    "JCAP",
    "JCE",
    "JCI",
    "JCO",
    "JCOM",
    "JCP",
    "JCS",
    "JCTCF",
    "JD",
    "JDD",
    "JDIV",
    "JE",
    "JEC",
    "JELD",
    "JEMD",
    "JEQ",
    "JETS",
    "JE_A",
    "JFR",
    "JGH",
    "JHA",
    "JHB",
    "JHD",
    "JHG",
    "JHI",
    "JHMA",
    "JHMC",
    "JHMD",
    "JHME",
    "JHMF",
    "JHMH",
    "JHMI",
    "JHML",
    "JHMM",
    "JHMS",
    "JHMT",
    "JHMU",
    "JHS",
    "JHSC",
    "JHX",
    "JHY",
    "JILL",
    "JJSF",
    "JKHY",
    "JKS",
    "JLL",
    "JLS",
    "JMBA",
    "JMEI",
    "JMF",
    "JMLP",
    "JMM",
    "JMOM",
    "JMP",
    "JMPB",
    "JMPC",
    "JMT",
    "JMU",
    "JNCE",
    "JNJ",
    "JNP",
    "JNPR",
    "JOB",
    "JOBS",
    "JOE",
    "JOF",
    "JONE",
    "JOUT",
    "JP",
    "JPC",
    "JPEH",
    "JPEM",
    "JPEU",
    "JPGB",
    "JPHF",
    "JPHY",
    "JPI",
    "JPIH",
    "JPIN",
    "JPM-WS",
    "JPM",
    "JPME",
    "JPM_A",
    "JPM_B",
    "JPM_D-CL",
    "JPM_D",
    "JPM_E",
    "JPM_F",
    "JPM_G",
    "JPM_H",
    "JPN",
    "JPS",
    "JPSE",
    "JPST",
    "JPT",
    "JPUS",
    "JPXN",
    "JQC",
    "JRI",
    "JRJC",
    "JRJR",
    "JRO",
    "JRS",
    "JRVR",
    "JSD",
    "JSM",
    "JSMD",
    "JSML",
    "JSYN",
    "JSYNR",
    "JSYNU",
    "JSYNW",
    "JT",
    "JTA",
    "JTD",
    "JTPY",
    "JUNO",
    "JVA",
    "JW-A",
    "JW-B",
    "JWN",
    "JXSB",
    "JYNT",
    "K",
    "KAAC",
    "KAACU",
    "KAACW",
    "KAI",
    "KALA",
    "KALU",
    "KALV",
    "KAMN",
    "KANG",
    "KAP",
    "KAR",
    "KB",
    "KBAL",
    "KBH",
    "KBLM",
    "KBLMR",
    "KBLMU",
    "KBLMW",
    "KBR",
    "KBSF",
    "KBWB",
    "KBWD",
    "KBWY",
    "KCAP",
    "KCAPL",
    "KCNY",
    "KDMN",
    "KE",
    "KED",
    "KEG",
    "KELYA",
    "KELYB",
    "KEM",
    "KEMP",
    "KEMQ",
    "KEN",
    "KEP",
    "KEQU",
    "KERX",
    "KEX",
    "KEY",
    "KEYS",
    "KEYW",
    "KEY_I",
    "KF",
    "KFFB",
    "KFN_",
    "KFRC",
    "KFS",
    "KFY",
    "KGC",
    "KGJI",
    "KGRN",
    "KHC",
    "KIDS",
    "KIM",
    "KIM_I",
    "KIM_J",
    "KIM_K",
    "KIM_L",
    "KIN",
    "KINS",
    "KIO",
    "KIOR",
    "KIORW",
    "KIQ",
    "KIRK",
    "KKR",
    "KKR_A",
    "KKR_B",
    "KL",
    "KLAC",
    "KLDW",
    "KLDX",
    "KLIC",
    "KLXI",
    "KMB",
    "KMDA",
    "KMF",
    "KMG",
    "KMI",
    "KMI_A",
    "KMM",
    "KMPA",
    "KMPH",
    "KMPR",
    "KMT",
    "KMX",
    "KN",
    "KND",
    "KNDI",
    "KNL",
    "KNOP",
    "KNSL",
    "KNX",
    "KO",
    "KODK-WS-A",
    "KODK-WS",
    "KODK",
    "KOF",
    "KONA",
    "KONE",
    "KOOL",
    "KOP",
    "KOPN",
    "KOR",
    "KORS",
    "KOS",
    "KOSS",
    "KPTI",
    "KR",
    "KRA",
    "KRC",
    "KREF",
    "KRG",
    "KRMA",
    "KRNT",
    "KRNY",
    "KRO",
    "KRP",
    "KRYS",
    "KS",
    "KSA",
    "KSM",
    "KSS",
    "KST",
    "KSU",
    "KSU_",
    "KT",
    "KTCC",
    "KTEC",
    "KTF",
    "KTH",
    "KTN",
    "KTOS",
    "KTOV",
    "KTOVW",
    "KTP",
    "KTWO",
    "KURA",
    "KVHI",
    "KW",
    "KWEB",
    "KWN-CL",
    "KWN",
    "KWR",
    "KYE",
    "KYN",
    "KYN_F",
    "KYO",
    "L",
    "LABL",
    "LAD",
    "LADR",
    "LAKE",
    "LAMR",
    "LANC",
    "LAND",
    "LANDP",
    "LAQ",
    "LARE",
    "LARK",
    "LAUR",
    "LAWS",
    "LAYN",
    "LAZ",
    "LB",
    "LBAI",
    "LBDC",
    "LBIX",
    "LBRDA",
    "LBRDK",
    "LBTYA",
    "LBTYB",
    "LBTYK",
    "LBY",
    "LC",
    "LCA",
    "LCAHU",
    "LCAHW",
    "LCI",
    "LCII",
    "LCM",
    "LCNB",
    "LCUT",
    "LDF",
    "LDL",
    "LDOS",
    "LDP",
    "LDR",
    "LE",
    "LEA",
    "LEAD",
    "LECO",
    "LEDS",
    "LEE",
    "LEG",
    "LEJU",
    "LEN-B",
    "LEN",
    "LENS",
    "LEO",
    "LEU",
    "LEXEA",
    "LEXEB",
    "LFC",
    "LFEQ",
    "LFGR",
    "LFUS",
    "LFVN",
    "LGCY",
    "LGCYO",
    "LGCYP",
    "LGF-A",
    "LGF-B",
    "LGI",
    "LGIH",
    "LGL",
    "LGLR",
    "LGND",
    "LH",
    "LHCG",
    "LHO",
    "LHO_I",
    "LHO_J",
    "LIFE",
    "LII",
    "LILA",
    "LILAK",
    "LINC",
    "LIND",
    "LINDW",
    "LINK",
    "LINU",
    "LION",
    "LIQT",
    "LITB",
    "LITE",
    "LIVE",
    "LIVN",
    "LJPC",
    "LKFN",
    "LKOR",
    "LKQ",
    "LKSD",
    "LL",
    "LLEX",
    "LLIT",
    "LLL",
    "LLNW",
    "LLQD",
    "LLY",
    "LM",
    "LMAT",
    "LMB",
    "LMFA",
    "LMFAW",
    "LMHA",
    "LMHB",
    "LMNR",
    "LMNX",
    "LMOS",
    "LMRK",
    "LMRKO",
    "LMRKP",
    "LMT",
    "LN",
    "LNC-WS",
    "LNC",
    "LNCE",
    "LND",
    "LNDC",
    "LNG",
    "LNGR",
    "LNN",
    "LNT",
    "LNTH",
    "LOAN",
    "LOB",
    "LOCO",
    "LODE",
    "LOGI",
    "LOGM",
    "LOGO",
    "LOMA",
    "LONE",
    "LOPE",
    "LOR",
    "LORL",
    "LOV",
    "LOVW",
    "LOW",
    "LOXO",
    "LPCN",
    "LPG",
    "LPI",
    "LPL",
    "LPLA",
    "LPNT",
    "LPSN",
    "LPT",
    "LPTH",
    "LPTX",
    "LPX",
    "LQ",
    "LQDT",
    "LRAD",
    "LRCX",
    "LRET",
    "LRGE",
    "LRN",
    "LSBK",
    "LSCC",
    "LSI",
    "LSTR",
    "LSVX",
    "LSXMA",
    "LSXMB",
    "LSXMK",
    "LTBR",
    "LTC",
    "LTEA",
    "LTM",
    "LTRPA",
    "LTRPB",
    "LTRX",
    "LTS",
    "LTS_A",
    "LTXB",
    "LUB",
    "LUK",
    "LULU",
    "LUNA",
    "LUV",
    "LVHB",
    "LVHD",
    "LVHE",
    "LVHI",
    "LVIN",
    "LVLT",
    "LVNTA",
    "LVNTB",
    "LVS",
    "LVUS",
    "LW",
    "LWAY",
    "LXFR",
    "LXFT",
    "LXP",
    "LXP_C",
    "LXRX",
    "LXU",
    "LYB",
    "LYG",
    "LYL",
    "LYTS",
    "LYV",
    "LZB",
    "M",
    "MA",
    "MAA",
    "MAA_I",
    "MAB",
    "MAC",
    "MACK",
    "MACQ",
    "MACQU",
    "MACQW",
    "MAG",
    "MAGA",
    "MAGS",
    "MAIN",
    "MAMS",
    "MAN",
    "MANH",
    "MANT",
    "MANU",
    "MAPI",
    "MAR",
    "MARA",
    "MARK",
    "MARPS",
    "MAS",
    "MASI",
    "MAT",
    "MATF",
    "MATR",
    "MATW",
    "MATX",
    "MAV",
    "MAXR",
    "MAYS",
    "MB",
    "MBCN",
    "MBFI",
    "MBFIP",
    "MBI",
    "MBII",
    "MBIN",
    "MBIO",
    "MBOT",
    "MBRX",
    "MBT",
    "MBTF",
    "MBUU",
    "MBVX",
    "MBWM",
    "MC",
    "MCA",
    "MCB",
    "MCBC",
    "MCC",
    "MCD",
    "MCEF",
    "MCEP",
    "MCF",
    "MCFT",
    "MCHP",
    "MCHX",
    "MCI",
    "MCK",
    "MCN",
    "MCO",
    "MCR",
    "MCRB",
    "MCRI",
    "MCRN",
    "MCS",
    "MCV",
    "MCX",
    "MCY",
    "MD",
    "MDB",
    "MDC",
    "MDCA",
    "MDCO",
    "MDGL",
    "MDGS",
    "MDLQ",
    "MDLX",
    "MDLY",
    "MDLZ",
    "MDP",
    "MDR",
    "MDRX",
    "MDSO",
    "MDT",
    "MDU",
    "MDVX",
    "MDVXW",
    "MDWD",
    "MDXG",
    "MEAR",
    "MED",
    "MEDP",
    "MEET",
    "MEI",
    "MEIP",
    "MELI",
    "MELR",
    "MEN",
    "MENU",
    "MEOH",
    "MERC",
    "MER_K",
    "MER_P",
    "MESO",
    "MET",
    "METC",
    "MET_A",
    "MEXX",
    "MFA",
    "MFA_B",
    "MFC",
    "MFCB",
    "MFDX",
    "MFEM",
    "MFG",
    "MFGP",
    "MFIN",
    "MFINL",
    "MFL",
    "MFM",
    "MFNC",
    "MFO",
    "MFSF",
    "MFT",
    "MFUS",
    "MFV",
    "MG",
    "MGA",
    "MGCD",
    "MGEE",
    "MGEN",
    "MGF",
    "MGI",
    "MGIC",
    "MGLN",
    "MGM",
    "MGNX",
    "MGP",
    "MGPI",
    "MGRC",
    "MGU",
    "MGYR",
    "MHD",
    "MHE",
    "MHF",
    "MHH",
    "MHI",
    "MHK",
    "MHLA",
    "MHLD",
    "MHN",
    "MHNC",
    "MHO",
    "MH_A",
    "MH_C",
    "MH_D",
    "MIC",
    "MICR",
    "MICT",
    "MICTW",
    "MIDD",
    "MIE",
    "MIII",
    "MIIIU",
    "MIIIW",
    "MIK",
    "MILN",
    "MIME",
    "MIN",
    "MIND",
    "MINDP",
    "MINI",
    "MITK",
    "MITL",
    "MITT",
    "MITT_A",
    "MITT_B",
    "MIW",
    "MIXT",
    "MIY",
    "MJCO",
    "MKC-V",
    "MKC",
    "MKL",
    "MKSI",
    "MKTX",
    "MLAB",
    "MLCO",
    "MLHR",
    "MLI",
    "MLM",
    "MLNK",
    "MLNT",
    "MLNX",
    "MLP",
    "MLPB",
    "MLPQ",
    "MLPZ",
    "MLQD",
    "MLR",
    "MLSS",
    "MLTI",
    "MLVF",
    "MMAC",
    "MMC",
    "MMD",
    "MMDM",
    "MMDMR",
    "MMDMU",
    "MMDMW",
    "MMI",
    "MMIT",
    "MMLP",
    "MMM",
    "MMP",
    "MMS",
    "MMSI",
    "MMT",
    "MMU",
    "MMV",
    "MMYT",
    "MN",
    "MNDO",
    "MNE",
    "MNGA",
    "MNI",
    "MNK",
    "MNKD",
    "MNOV",
    "MNP",
    "MNR",
    "MNRO",
    "MNR_C",
    "MNST",
    "MNTA",
    "MNTX",
    "MO",
    "MOBL",
    "MOC",
    "MOD",
    "MODN",
    "MOFG",
    "MOG-A",
    "MOG-B",
    "MOGLC",
    "MOH",
    "MOLC",
    "MOMO",
    "MON",
    "MORN",
    "MOS",
    "MOSC-U",
    "MOSY",
    "MOTI",
    "MOV",
    "MOXC",
    "MPA",
    "MPAA",
    "MPAC",
    "MPACU",
    "MPACW",
    "MPB",
    "MPC",
    "MPCT",
    "MPLX",
    "MPO",
    "MPV",
    "MPVD",
    "MPW",
    "MPWR",
    "MPX",
    "MP_D",
    "MQT",
    "MQY",
    "MRAM",
    "MRBK",
    "MRC",
    "MRCC",
    "MRCY",
    "MRDN",
    "MRDNW",
    "MRIN",
    "MRK",
    "MRLN",
    "MRNS",
    "MRO",
    "MRRL",
    "MRSN",
    "MRT",
    "MRTN",
    "MRTX",
    "MRUS",
    "MRVL",
    "MS",
    "MSA",
    "MSB",
    "MSBF",
    "MSBI",
    "MSCA",
    "MSCC",
    "MSCI",
    "MSD",
    "MSDI",
    "MSDIW",
    "MSEX",
    "MSF",
    "MSFG",
    "MSFT",
    "MSG",
    "MSGN",
    "MSI",
    "MSL",
    "MSM",
    "MSN",
    "MSON",
    "MSP",
    "MSTR",
    "MS_A",
    "MS_E",
    "MS_F",
    "MS_G",
    "MS_I",
    "MS_K",
    "MT",
    "MTB-WS",
    "MTB",
    "MTBC",
    "MTBCP",
    "MTB_",
    "MTB_C",
    "MTCH",
    "MTD",
    "MTDR",
    "MTEM",
    "MTEX",
    "MTFB",
    "MTFBW",
    "MTG",
    "MTGE",
    "MTGEP",
    "MTH",
    "MTL",
    "MTLS",
    "MTL_",
    "MTN",
    "MTNB",
    "MTOR",
    "MTP",
    "MTR",
    "MTRN",
    "MTRX",
    "MTSC",
    "MTSI",
    "MTSL",
    "MTT",
    "MTU",
    "MTW",
    "MTX",
    "MTZ",
    "MU",
    "MUA",
    "MUC",
    "MUE",
    "MUH",
    "MUI",
    "MUJ",
    "MULE",
    "MUR",
    "MUS",
    "MUSA",
    "MUX",
    "MVC",
    "MVCB",
    "MVF",
    "MVIN",
    "MVIS",
    "MVO",
    "MVT",
    "MWA",
    "MX",
    "MXC",
    "MXDU",
    "MXE",
    "MXF",
    "MXIM",
    "MXL",
    "MXWL",
    "MYC",
    "MYD",
    "MYE",
    "MYF",
    "MYGN",
    "MYI",
    "MYJ",
    "MYL",
    "MYN",
    "MYND",
    "MYNDW",
    "MYO",
    "MYOK",
    "MYOS",
    "MYOV",
    "MYRG",
    "MYSZ",
    "MZA",
    "MZF",
    "MZOR",
    "NAC",
    "NAD",
    "NAII",
    "NAIL",
    "NAK",
    "NAKD",
    "NAN",
    "NANO",
    "NANR",
    "NAO",
    "NAOV",
    "NAP",
    "NAT",
    "NATH",
    "NATI",
    "NATR",
    "NAUH",
    "NAV",
    "NAVB",
    "NAVG",
    "NAVI",
    "NAV_D",
    "NAZ",
    "NBB",
    "NBD",
    "NBEV",
    "NBH",
    "NBHC",
    "NBIX",
    "NBL",
    "NBLX",
    "NBN",
    "NBO",
    "NBR",
    "NBRV",
    "NBTB",
    "NBW",
    "NBY",
    "NC",
    "NCA",
    "NCB",
    "NCBS",
    "NCI",
    "NCLH",
    "NCMI",
    "NCNA",
    "NCOM",
    "NCR",
    "NCS",
    "NCSM",
    "NCTY",
    "NCV",
    "NCZ",
    "NDAQ",
    "NDLS",
    "NDP",
    "NDRA",
    "NDRAW",
    "NDRM",
    "NDRO",
    "NDSN",
    "NE",
    "NEA",
    "NEE",
    "NEE_C-CL",
    "NEE_C",
    "NEE_G-CL",
    "NEE_H-CL",
    "NEE_I",
    "NEE_J",
    "NEE_K",
    "NEE_Q",
    "NEE_R",
    "NEM",
    "NEN",
    "NEO",
    "NEOG",
    "NEON",
    "NEOS",
    "NEOT",
    "NEP",
    "NEPT",
    "NERV",
    "NES",
    "NESR",
    "NESRW",
    "NETE",
    "NETS",
    "NEU",
    "NEV",
    "NEWA",
    "NEWM",
    "NEWR",
    "NEWS",
    "NEWT",
    "NEWTL",
    "NEWTZ",
    "NEXA",
    "NEXT",
    "NEXTW",
    "NFBK",
    "NFEC",
    "NFG",
    "NFJ",
    "NFLT",
    "NFLX",
    "NFX",
    "NG",
    "NGD",
    "NGG",
    "NGHC",
    "NGHCN",
    "NGHCO",
    "NGHCP",
    "NGHCZ",
    "NGL",
    "NGLS_A",
    "NGL_B",
    "NGS",
    "NGVC",
    "NGVT",
    "NH",
    "NHA",
    "NHC",
    "NHF",
    "NHI",
    "NHLD",
    "NHLDW",
    "NHS",
    "NHTC",
    "NI",
    "NICE",
    "NICK",
    "NID",
    "NIE",
    "NIHD",
    "NIM",
    "NIQ",
    "NITE",
    "NJR",
    "NJV",
    "NK",
    "NKE",
    "NKG",
    "NKSH",
    "NKTR",
    "NKX",
    "NL",
    "NLNK",
    "NLS",
    "NLSN",
    "NLST",
    "NLY",
    "NLY_C",
    "NLY_D",
    "NLY_E",
    "NLY_F",
    "NM",
    "NMFC",
    "NMI",
    "NMIH",
    "NMK_B",
    "NMK_C",
    "NML",
    "NMM",
    "NMR",
    "NMRX",
    "NMS",
    "NMT",
    "NMY",
    "NMZ",
    "NM_G",
    "NM_H",
    "NNA",
    "NNBR",
    "NNC",
    "NNDM",
    "NNI",
    "NNN",
    "NNN_E",
    "NNN_F",
    "NNVC",
    "NNY",
    "NOA",
    "NOAH",
    "NOC",
    "NODK",
    "NOG",
    "NOK",
    "NOM",
    "NOMD",
    "NOV",
    "NOVN",
    "NOVT",
    "NOW",
    "NP",
    "NPK",
    "NPN",
    "NPO",
    "NPTN",
    "NPV",
    "NQ",
    "NQP",
    "NR",
    "NRCIA",
    "NRCIB",
    "NRE",
    "NRG",
    "NRIM",
    "NRK",
    "NRO",
    "NRP",
    "NRT",
    "NRZ",
    "NS",
    "NSA",
    "NSA_A",
    "NSC",
    "NSEC",
    "NSH",
    "NSIT",
    "NSL",
    "NSM",
    "NSP",
    "NSPR-WS-B",
    "NSPR-WS",
    "NSPR",
    "NSS",
    "NSSC",
    "NSTG",
    "NSU",
    "NSYS",
    "NS_A",
    "NS_B",
    "NTAP",
    "NTB",
    "NTC",
    "NTCT",
    "NTEC",
    "NTES",
    "NTG",
    "NTGR",
    "NTIC",
    "NTIP",
    "NTL",
    "NTLA",
    "NTN",
    "NTNX",
    "NTP",
    "NTRA",
    "NTRI",
    "NTRP",
    "NTRS",
    "NTRSP",
    "NTWK",
    "NTX",
    "NTZ",
    "NUAG",
    "NUAN",
    "NUBD",
    "NUDM",
    "NUE",
    "NUEM",
    "NULG",
    "NULV",
    "NUM",
    "NUMG",
    "NUMV",
    "NUO",
    "NURE",
    "NURO",
    "NUROW",
    "NUS",
    "NUSA",
    "NUSC",
    "NUV",
    "NUVA",
    "NUW",
    "NVAX",
    "NVCN",
    "NVCR",
    "NVDA",
    "NVEC",
    "NVEE",
    "NVFY",
    "NVG",
    "NVGN",
    "NVGS",
    "NVIV",
    "NVLN",
    "NVMI",
    "NVO",
    "NVR",
    "NVRO",
    "NVS",
    "NVTA",
    "NVTR",
    "NVUS",
    "NWBI",
    "NWE",
    "NWFL",
    "NWHM",
    "NWL",
    "NWLI",
    "NWN",
    "NWPX",
    "NWS",
    "NWSA",
    "NWY",
    "NW_C-CL",
    "NX",
    "NXC",
    "NXE",
    "NXEO",
    "NXEOU",
    "NXEOW",
    "NXJ",
    "NXN",
    "NXP",
    "NXPI",
    "NXQ",
    "NXR",
    "NXRT",
    "NXST",
    "NXTD",
    "NXTDW",
    "NXTM",
    "NYCB",
    "NYCB_A",
    "NYCB_U",
    "NYH",
    "NYLD-A",
    "NYLD",
    "NYMT",
    "NYMTN",
    "NYMTO",
    "NYMTP",
    "NYMX",
    "NYNY",
    "NYRT",
    "NYT",
    "NYV",
    "NZF",
    "O",
    "OA",
    "OACQ",
    "OACQR",
    "OACQU",
    "OACQW",
    "OAK",
    "OAKS",
    "OAKS_A",
    "OAPH",
    "OAS",
    "OASI",
    "OASM",
    "OBAS",
    "OBCI",
    "OBE",
    "OBLN",
    "OBOR",
    "OBSV",
    "OC",
    "OCC",
    "OCFC",
    "OCIO",
    "OCIP",
    "OCLR",
    "OCN",
    "OCRX",
    "OCSI",
    "OCSL",
    "OCSLL",
    "OCUL",
    "OCX",
    "ODC",
    "ODFL",
    "ODP",
    "OEC",
    "OESX",
    "OEUH",
    "OEUR",
    "OEW",
    "OFC",
    "OFED",
    "OFG",
    "OFG_A",
    "OFG_B",
    "OFG_D",
    "OFIX",
    "OFLX",
    "OFS",
    "OGCP",
    "OGE",
    "OGEN",
    "OGS",
    "OHAI",
    "OHGI",
    "OHI",
    "OHRP",
    "OI",
    "OIA",
    "OIBR-C",
    "OII",
    "OIIL",
    "OIIM",
    "OILB",
    "OILD",
    "OILK",
    "OILU",
    "OILX",
    "OIS",
    "OKE",
    "OKSB",
    "OKTA",
    "OLBK",
    "OLD",
    "OLED",
    "OLLI",
    "OLN",
    "OLP",
    "OMAA",
    "OMAB",
    "OMAM",
    "OMC",
    "OMCL",
    "OME",
    "OMED",
    "OMER",
    "OMEX",
    "OMF",
    "OMFL",
    "OMFS",
    "OMI",
    "OMN",
    "OMNT",
    "OMP",
    "ON",
    "ONB",
    "ONCE",
    "ONCS",
    "ONDK",
    "ONEO",
    "ONEV",
    "ONEY",
    "ONP",
    "ONS",
    "ONSIW",
    "ONSIZ",
    "ONTL",
    "ONTX",
    "ONTXW",
    "ONVI",
    "ONVO",
    "OOMA",
    "OPB",
    "OPGN",
    "OPGNW",
    "OPHC",
    "OPHT",
    "OPK",
    "OPNT",
    "OPOF",
    "OPP",
    "OPTN",
    "OPTT",
    "OPY",
    "OR",
    "ORA",
    "ORAN",
    "ORBC",
    "ORBK",
    "ORC",
    "ORCL",
    "OREX",
    "ORG",
    "ORI",
    "ORIG",
    "ORIT",
    "ORLY",
    "ORM",
    "ORMP",
    "ORN",
    "ORPN",
    "ORRF",
    "OSB",
    "OSBC",
    "OSBCP",
    "OSG",
    "OSIS",
    "OSK",
    "OSLE",
    "OSN",
    "OSPR",
    "OSPRU",
    "OSPRW",
    "OSTK",
    "OSUR",
    "OTEL",
    "OTEX",
    "OTG",
    "OTIC",
    "OTIV",
    "OTTR",
    "OTTW",
    "OUSA",
    "OUSM",
    "OUT",
    "OVAS",
    "OVBC",
    "OVID",
    "OVLY",
    "OXBR",
    "OXBRW",
    "OXFD",
    "OXLC",
    "OXLCM",
    "OXLCO",
    "OXM",
    "OXY",
    "OZM",
    "OZRK",
    "P",
    "PAA",
    "PAAS",
    "PAC",
    "PACB",
    "PACW",
    "PAG",
    "PAGP",
    "PAH",
    "PAHC",
    "PAI",
    "PAM",
    "PANL",
    "PANW",
    "PAR",
    "PARR",
    "PATI",
    "PATK",
    "PAVE",
    "PAVM",
    "PAVMW",
    "PAY",
    "PAYC",
    "PAYX",
    "PB",
    "PBA",
    "PBB",
    "PBBI",
    "PBCT",
    "PBCTP",
    "PBDM",
    "PBEE",
    "PBF",
    "PBFX",
    "PBH",
    "PBHC",
    "PBI",
    "PBIB",
    "PBIO",
    "PBIP",
    "PBI_B",
    "PBMD",
    "PBNC",
    "PBND",
    "PBPB",
    "PBR-A",
    "PBR",
    "PBSK",
    "PBSM",
    "PBT",
    "PBUS",
    "PBYI",
    "PCAR",
    "PCBK",
    "PCF",
    "PCG",
    "PCG_A",
    "PCG_B",
    "PCG_C",
    "PCG_D",
    "PCG_E",
    "PCG_G",
    "PCG_H",
    "PCG_I",
    "PCH",
    "PCI",
    "PCK",
    "PCLN",
    "PCM",
    "PCMI",
    "PCN",
    "PCO",
    "PCOM",
    "PCQ",
    "PCRX",
    "PCSB",
    "PCTI",
    "PCTY",
    "PCYG",
    "PCYO",
    "PDCE",
    "PDCO",
    "PDEX",
    "PDFS",
    "PDI",
    "PDLB",
    "PDLI",
    "PDM",
    "PDP",
    "PDS",
    "PDT",
    "PDVW",
    "PE",
    "PEB",
    "PEBK",
    "PEBO",
    "PEB_C",
    "PEB_D",
    "PED",
    "PEG",
    "PEGA",
    "PEGI",
    "PEI",
    "PEIX",
    "PEI_B",
    "PEI_C",
    "PEI_D",
    "PEN",
    "PENN",
    "PEO",
    "PEP",
    "PER",
    "PERI",
    "PERY",
    "PES",
    "PESI",
    "PETQ",
    "PETS",
    "PETX",
    "PETZ",
    "PF",
    "PFBC",
    "PFBI",
    "PFBX",
    "PFD",
    "PFE",
    "PFFD",
    "PFFR",
    "PFG",
    "PFGC",
    "PFH",
    "PFI",
    "PFIE",
    "PFIN",
    "PFIS",
    "PFK",
    "PFL",
    "PFLT",
    "PFMT",
    "PFN",
    "PFNX",
    "PFO",
    "PFPT",
    "PFS",
    "PFSI",
    "PFSW",
    "PG",
    "PGC",
    "PGEM",
    "PGH",
    "PGLC",
    "PGNX",
    "PGP",
    "PGR",
    "PGRE",
    "PGTI",
    "PGZ",
    "PH",
    "PHD",
    "PHG",
    "PHH",
    "PHI",
    "PHII",
    "PHIIK",
    "PHK",
    "PHM",
    "PHO",
    "PHT",
    "PHX",
    "PHYS",
    "PI",
    "PICO",
    "PID",
    "PIH",
    "PII",
    "PIM",
    "PINC",
    "PIR",
    "PIRS",
    "PIXY",
    "PIY",
    "PIZ",
    "PJC",
    "PJH",
    "PJT",
    "PK",
    "PKBK",
    "PKD",
    "PKE",
    "PKG",
    "PKI",
    "PKO",
    "PKOH",
    "PKW",
    "PKX",
    "PKY",
    "PLAB",
    "PLAY",
    "PLBC",
    "PLCE",
    "PLD",
    "PLG",
    "PLM",
    "PLNT",
    "PLOW",
    "PLPC",
    "PLPM",
    "PLSE",
    "PLT",
    "PLUG",
    "PLUS",
    "PLW",
    "PLX",
    "PLXP",
    "PLXS",
    "PLYA",
    "PLYM",
    "PLYM_A",
    "PM",
    "PMBC",
    "PMC",
    "PMD",
    "PME",
    "PMF",
    "PML",
    "PMM",
    "PMO",
    "PMOM",
    "PMPT",
    "PMT",
    "PMTS",
    "PMT_A",
    "PMT_B",
    "PMX",
    "PN",
    "PNBK",
    "PNC-WS",
    "PNC",
    "PNC_P",
    "PNC_Q",
    "PNF",
    "PNFP",
    "PNI",
    "PNK",
    "PNM",
    "PNNT",
    "PNR",
    "PNRG",
    "PNTR",
    "PNW",
    "PODD",
    "POL",
    "POLA",
    "POOL",
    "POPE",
    "POR",
    "POST",
    "POT",
    "POWI",
    "POWL",
    "PPBI",
    "PPC",
    "PPDF",
    "PPG",
    "PPHM",
    "PPHMP",
    "PPIH",
    "PPL",
    "PPLN",
    "PPR",
    "PPSI",
    "PPT",
    "PPX",
    "PQ",
    "PQG",
    "PRA",
    "PRAA",
    "PRAH",
    "PRAN",
    "PRCP",
    "PREF",
    "PRE_F",
    "PRE_G",
    "PRE_H",
    "PRE_I",
    "PRFT",
    "PRGO",
    "PRGS",
    "PRGX",
    "PRH",
    "PRI",
    "PRIM",
    "PRK",
    "PRKR",
    "PRLB",
    "PRME",
    "PRMW",
    "PRNT",
    "PRO",
    "PROV",
    "PRPH",
    "PRPO",
    "PRQR",
    "PRSC",
    "PRSS",
    "PRTA",
    "PRTK",
    "PRTO",
    "PRTS",
    "PRTY",
    "PRU",
    "PSA",
    "PSA_A",
    "PSA_B",
    "PSA_C",
    "PSA_D",
    "PSA_E",
    "PSA_F",
    "PSA_G",
    "PSA_U",
    "PSA_V",
    "PSA_W",
    "PSA_X",
    "PSA_Y",
    "PSA_Z",
    "PSB",
    "PSB_T",
    "PSB_U",
    "PSB_V",
    "PSB_W",
    "PSB_X",
    "PSC",
    "PSDO",
    "PSDV",
    "PSEC",
    "PSET",
    "PSF",
    "PSL",
    "PSLV",
    "PSMB",
    "PSMC",
    "PSMG",
    "PSMM",
    "PSMT",
    "PSO",
    "PSTB",
    "PSTG",
    "PSTI",
    "PSX",
    "PSXP",
    "PTC",
    "PTCT",
    "PTEN",
    "PTEU",
    "PTF",
    "PTGX",
    "PTH",
    "PTI",
    "PTIE",
    "PTLA",
    "PTLC",
    "PTMC",
    "PTN",
    "PTNQ",
    "PTNR",
    "PTR",
    "PTSI",
    "PTX",
    "PTY",
    "PUB",
    "PUI",
    "PUK",
    "PUK_",
    "PUK_A",
    "PULM",
    "PUMP",
    "PUTW",
    "PVAC",
    "PVAL",
    "PVBC",
    "PVG",
    "PVH",
    "PW",
    "PWOD",
    "PWR",
    "PW_A",
    "PX",
    "PXD",
    "PXI",
    "PXLW",
    "PXS",
    "PXUS",
    "PY",
    "PYDS",
    "PYN",
    "PYPL",
    "PYS",
    "PYT",
    "PYZ",
    "PZC",
    "PZE",
    "PZG",
    "PZN",
    "PZRX",
    "PZZA",
    "Q",
    "QADA",
    "QADB",
    "QBAK",
    "QCOM",
    "QCP",
    "QCRH",
    "QD",
    "QDEL",
    "QEP",
    "QGEN",
    "QGTA",
    "QHC",
    "QIWI",
    "QLC",
    "QLYS",
    "QMOM",
    "QNST",
    "QQQX",
    "QRHC",
    "QRVO",
    "QSII",
    "QSR",
    "QTM",
    "QTNA",
    "QTNT",
    "QTRH",
    "QTS",
    "QTWO",
    "QUAD",
    "QUIK",
    "QUMU",
    "QUOT",
    "QURE",
    "QVAL",
    "QVCA",
    "QVCB",
    "QVM",
    "QXGG",
    "QXMI",
    "QXRR",
    "QXTR",
    "R",
    "RA",
    "RACE",
    "RAD",
    "RADA",
    "RAIL",
    "RAND",
    "RARE",
    "RARX",
    "RAS",
    "RAS_A",
    "RAS_B",
    "RAS_C",
    "RATE",
    "RAVE",
    "RAVN",
    "RBA",
    "RBB",
    "RBC",
    "RBCAA",
    "RBCN",
    "RBIN",
    "RBIO",
    "RBPAA",
    "RBS",
    "RBS_S",
    "RBUS",
    "RCG",
    "RCI",
    "RCII",
    "RCKY",
    "RCL",
    "RCM",
    "RCMT",
    "RCOM",
    "RCON",
    "RCS",
    "RDC",
    "RDCM",
    "RDFN",
    "RDHL",
    "RDI",
    "RDIB",
    "RDN",
    "RDNT",
    "RDS-A",
    "RDS-B",
    "RDUS",
    "RDWR",
    "RDY",
    "RE",
    "RECN",
    "REDU",
    "REED",
    "REEM",
    "REFA",
    "REFR",
    "REG",
    "REGI",
    "REGN",
    "REI",
    "REIS",
    "RELL",
    "RELV",
    "RELX",
    "RELY",
    "REML",
    "REN",
    "RENN",
    "RENX",
    "REPH",
    "RES",
    "RESI",
    "RESN",
    "RETA",
    "REV",
    "REVG",
    "REX",
    "REXR",
    "REXR_A",
    "REXX",
    "RF",
    "RFAP",
    "RFCI",
    "RFDA",
    "RFDI",
    "RFEM",
    "RFEU",
    "RFFC",
    "RFI",
    "RFIL",
    "RFP",
    "RFT",
    "RFTA",
    "RFUN",
    "RF_A",
    "RF_B",
    "RGA",
    "RGC",
    "RGCO",
    "RGEN",
    "RGLB",
    "RGLD",
    "RGLS",
    "RGNX",
    "RGR",
    "RGS",
    "RGSE",
    "RGT",
    "RH",
    "RHE",
    "RHE_A",
    "RHI",
    "RHP",
    "RHT",
    "RIBT",
    "RIBTW",
    "RIC",
    "RICE",
    "RICK",
    "RIF",
    "RIG",
    "RIGL",
    "RILY",
    "RILYL",
    "RILYZ",
    "RIO",
    "RIOT",
    "RISE",
    "RIV",
    "RIVR",
    "RIVRW",
    "RJF",
    "RKDA",
    "RL",
    "RLGT",
    "RLGT_A",
    "RLGY",
    "RLH",
    "RLI",
    "RLJ",
    "RLJE",
    "RLJ_A",
    "RLOG",
    "RM",
    "RMAX",
    "RMBL",
    "RMBS",
    "RMCF",
    "RMD",
    "RMGN",
    "RMNI",
    "RMNIU",
    "RMNIW",
    "RMP",
    "RMPL_",
    "RMR",
    "RMT",
    "RMTI",
    "RNDB",
    "RNDM",
    "RNDV",
    "RNEM",
    "RNET",
    "RNG",
    "RNGR",
    "RNLC",
    "RNMC",
    "RNN",
    "RNP",
    "RNR",
    "RNR_C",
    "RNR_E",
    "RNSC",
    "RNST",
    "RNVA",
    "RNVAZ",
    "RNWK",
    "ROAM",
    "ROCK",
    "RODM",
    "ROG",
    "ROGS",
    "ROIC",
    "ROK",
    "ROKA",
    "ROKU",
    "ROL",
    "ROLL",
    "ROP",
    "RORE",
    "ROSE",
    "ROSEU",
    "ROSEW",
    "ROSG",
    "ROST",
    "ROUS",
    "ROX",
    "ROYT",
    "RP",
    "RPAI",
    "RPAI_A-CL",
    "RPAI_A",
    "RPD",
    "RPM",
    "RPRX",
    "RPT",
    "RPT_D",
    "RPXC",
    "RQI",
    "RRC",
    "RRD",
    "RRGB",
    "RRR",
    "RRTS",
    "RS",
    "RSG",
    "RSLS",
    "RSO",
    "RSO_A",
    "RSO_B",
    "RSO_C",
    "RSPP",
    "RST",
    "RSYS",
    "RT",
    "RTEC",
    "RTIX",
    "RTK",
    "RTN",
    "RTNB",
    "RTRX",
    "RTTR",
    "RUBI",
    "RUN",
    "RUSHA",
    "RUSHB",
    "RUTH",
    "RVEN",
    "RVLT",
    "RVNC",
    "RVP",
    "RVRS",
    "RVSB",
    "RVT",
    "RWC",
    "RWLK",
    "RWT",
    "RXDX",
    "RXII",
    "RXIIW",
    "RXN",
    "RXN_A",
    "RY",
    "RYAAY",
    "RYAM",
    "RYAM_A",
    "RYB",
    "RYI",
    "RYN",
    "RYTM",
    "RY_S-CL",
    "RY_T",
    "RZA",
    "RZB",
    "S",
    "SA",
    "SAB",
    "SABR",
    "SACH",
    "SAEX",
    "SAFE",
    "SAFM",
    "SAFT",
    "SAGE",
    "SAH",
    "SAIA",
    "SAIC",
    "SAIL",
    "SAL",
    "SALM",
    "SALT",
    "SAM",
    "SAMG",
    "SAN",
    "SAND",
    "SANM",
    "SANW",
    "SAN_A",
    "SAN_B",
    "SAN_C",
    "SAN_I",
    "SAP",
    "SAR",
    "SASR",
    "SATS",
    "SAUC",
    "SAVE",
    "SB",
    "SBAC",
    "SBBC",
    "SBBP",
    "SBBX",
    "SBCF",
    "SBCP",
    "SBFG",
    "SBFGP",
    "SBGI",
    "SBGL",
    "SBH",
    "SBI",
    "SBLK",
    "SBLKL",
    "SBNA",
    "SBNB",
    "SBNY",
    "SBNYW",
    "SBOT",
    "SBOW",
    "SBPH",
    "SBR",
    "SBRA",
    "SBRAP",
    "SBS",
    "SBSI",
    "SBT",
    "SBUX",
    "SB_B",
    "SB_C",
    "SB_D",
    "SC",
    "SCA",
    "SCAC",
    "SCACU",
    "SCACW",
    "SCAP",
    "SCCI",
    "SCCO",
    "SCD",
    "SCE_B",
    "SCE_C",
    "SCE_D",
    "SCE_E",
    "SCE_G",
    "SCE_H",
    "SCE_J",
    "SCE_K",
    "SCE_L",
    "SCG",
    "SCHK",
    "SCHL",
    "SCHN",
    "SCHW",
    "SCHW_B-CL",
    "SCHW_B",
    "SCHW_C",
    "SCHW_D",
    "SCI",
    "SCKT",
    "SCL",
    "SCLN",
    "SCM",
    "SCMP",
    "SCON",
    "SCPH",
    "SCS",
    "SCSC",
    "SCVL",
    "SCWX",
    "SCX",
    "SCYX",
    "SD",
    "SDLP",
    "SDPI",
    "SDR",
    "SDRL",
    "SDT",
    "SDVY",
    "SE",
    "SEAC",
    "SEAS",
    "SEB",
    "SECO",
    "SECT",
    "SEDG",
    "SEE",
    "SEED",
    "SEIC",
    "SELB",
    "SELF",
    "SEM",
    "SEMG",
    "SEND",
    "SENEA",
    "SENEB",
    "SENS",
    "SEP",
    "SERV",
    "SF",
    "SFB",
    "SFBC",
    "SFBS",
    "SFE",
    "SFHY",
    "SFIG",
    "SFIX",
    "SFL",
    "SFLY",
    "SFM",
    "SFNC",
    "SFR",
    "SFS",
    "SFST",
    "SFUN",
    "SF_A",
    "SGA",
    "SGB",
    "SGBX",
    "SGC",
    "SGEN",
    "SGF",
    "SGH",
    "SGLB",
    "SGLBW",
    "SGMA",
    "SGMO",
    "SGMS",
    "SGOC",
    "SGQI",
    "SGRP",
    "SGRY",
    "SGU",
    "SGY-WS",
    "SGY",
    "SGYP",
    "SGZA",
    "SHAG",
    "SHAK",
    "SHBI",
    "SHE",
    "SHEN",
    "SHG",
    "SHI",
    "SHIP",
    "SHIPW",
    "SHLD",
    "SHLDW",
    "SHLM",
    "SHLO",
    "SHLX",
    "SHNY",
    "SHO",
    "SHOO",
    "SHOP",
    "SHOS",
    "SHO_E",
    "SHO_F",
    "SHPG",
    "SHSP",
    "SHW",
    "SID",
    "SIEB",
    "SIEN",
    "SIF",
    "SIFI",
    "SIFY",
    "SIG",
    "SIGI",
    "SIGM",
    "SILC",
    "SIM",
    "SIMO",
    "SINA",
    "SINO",
    "SIR",
    "SIRI",
    "SITE",
    "SITO",
    "SIVB",
    "SIVBO",
    "SIX",
    "SJI",
    "SJM",
    "SJR",
    "SJT",
    "SJW",
    "SKIS",
    "SKLN",
    "SKM",
    "SKT",
    "SKX",
    "SKY",
    "SKYS",
    "SKYW",
    "SLAB",
    "SLB",
    "SLCA",
    "SLCT",
    "SLD",
    "SLDA",
    "SLF",
    "SLG",
    "SLGN",
    "SLG_I",
    "SLIM",
    "SLM",
    "SLMBP",
    "SLNO",
    "SLNOW",
    "SLP",
    "SLRA",
    "SLRC",
    "SLTB",
    "SM",
    "SMBC",
    "SMBK",
    "SMCI",
    "SMCP",
    "SMED",
    "SMFG",
    "SMG",
    "SMHD",
    "SMHI",
    "SMI",
    "SMIN",
    "SMIT",
    "SMLP",
    "SMM",
    "SMMD",
    "SMMF",
    "SMMT",
    "SMMV",
    "SMP",
    "SMPL",
    "SMPLW",
    "SMRT",
    "SMSI",
    "SMTC",
    "SMTS",
    "SMTX",
    "SN",
    "SNA",
    "SNAK",
    "SNAP",
    "SNBC",
    "SNBR",
    "SNC",
    "SNCR",
    "SND",
    "SNDE",
    "SNDR",
    "SNDX",
    "SNE",
    "SNES",
    "SNFCA",
    "SNGX",
    "SNGXW",
    "SNH",
    "SNHNI",
    "SNHNL",
    "SNHY",
    "SNI",
    "SNMP",
    "SNMX",
    "SNN",
    "SNNA",
    "SNOA",
    "SNOAW",
    "SNP",
    "SNPS",
    "SNR",
    "SNSR",
    "SNSS",
    "SNV",
    "SNV_C",
    "SNX",
    "SNY",
    "SO",
    "SODA",
    "SOFO",
    "SOGO",
    "SOHO",
    "SOHOB",
    "SOHOM",
    "SOHOO",
    "SOHU",
    "SOI",
    "SOJA",
    "SOJB",
    "SOL",
    "SON",
    "SONA",
    "SONC",
    "SONS",
    "SOR",
    "SORL",
    "SOVB",
    "SOV_C",
    "SP",
    "SPA",
    "SPAB",
    "SPAR",
    "SPB",
    "SPCB",
    "SPDN",
    "SPDW",
    "SPE",
    "SPEM",
    "SPEX",
    "SPE_B",
    "SPG",
    "SPGI",
    "SPG_J",
    "SPH",
    "SPHS",
    "SPI",
    "SPIB",
    "SPIL",
    "SPKE",
    "SPKEP",
    "SPLB",
    "SPLG",
    "SPLK",
    "SPLP",
    "SPLP_A",
    "SPLP_T",
    "SPMD",
    "SPMO",
    "SPMV",
    "SPN",
    "SPNE",
    "SPNS",
    "SPOK",
    "SPPI",
    "SPPP",
    "SPR",
    "SPRO",
    "SPRT",
    "SPSB",
    "SPSC",
    "SPSM",
    "SPTL",
    "SPTM",
    "SPTN",
    "SPTS",
    "SPUN",
    "SPVM",
    "SPVU",
    "SPWH",
    "SPWR",
    "SPXC",
    "SPXE",
    "SPXN",
    "SPXT",
    "SPXV",
    "SPXX",
    "SPYD",
    "SPYX",
    "SQ",
    "SQBG",
    "SQLV",
    "SQM",
    "SQNS",
    "SQZZ",
    "SR",
    "SRAX",
    "SRC",
    "SRCE",
    "SRCI",
    "SRCL",
    "SRCLP",
    "SRC_A",
    "SRDX",
    "SRE",
    "SREV",
    "SRF",
    "SRG",
    "SRI",
    "SRLP",
    "SRNE",
    "SRPT",
    "SRRA",
    "SRT",
    "SRTS",
    "SRTSW",
    "SRUN",
    "SRUNU",
    "SRUNW",
    "SRV",
    "SRVA",
    "SSB",
    "SSBI",
    "SSC",
    "SSD",
    "SSFN",
    "SSI",
    "SSKN",
    "SSL",
    "SSN",
    "SSNC",
    "SSNI",
    "SSNT",
    "SSP",
    "SSRM",
    "SSTI",
    "SSTK",
    "SSW",
    "SSWA",
    "SSWN",
    "SSW_D",
    "SSW_E",
    "SSW_G",
    "SSW_H",
    "SSY",
    "SSYS",
    "ST",
    "STAA",
    "STAF",
    "STAG",
    "STAG_B",
    "STAG_C",
    "STAR",
    "STAR_D",
    "STAR_E",
    "STAR_F",
    "STAR_G",
    "STAR_I",
    "STAY",
    "STB",
    "STBA",
    "STBZ",
    "STC",
    "STDY",
    "STE",
    "STFC",
    "STI-WS-A",
    "STI-WS-B",
    "STI",
    "STI_A",
    "STI_E",
    "STK",
    "STKL",
    "STKS",
    "STL",
    "STLD",
    "STLR",
    "STLRU",
    "STLRW",
    "STLY",
    "STL_A",
    "STM",
    "STML",
    "STMP",
    "STN",
    "STNG",
    "STNL",
    "STNLU",
    "STO",
    "STON",
    "STOR",
    "STOT",
    "STPP",
    "STRA",
    "STRL",
    "STRM",
    "STRP",
    "STRS",
    "STRT",
    "STT",
    "STT_C",
    "STT_D",
    "STT_E",
    "STT_G",
    "STWD",
    "STX",
    "STZ-B",
    "STZ",
    "SU",
    "SUI",
    "SUI_A",
    "SUM",
    "SUMR",
    "SUN",
    "SUNS",
    "SUNW",
    "SUP",
    "SUPN",
    "SUPV",
    "SUSA",
    "SUSB",
    "SUSC",
    "SVA",
    "SVBI",
    "SVM",
    "SVRA",
    "SVT",
    "SVU",
    "SVVC",
    "SWCH",
    "SWIN",
    "SWIR",
    "SWJ",
    "SWK",
    "SWKS",
    "SWM",
    "SWN",
    "SWNC",
    "SWP",
    "SWX",
    "SWZ",
    "SXC",
    "SXCP",
    "SXE",
    "SXI",
    "SXT",
    "SYBT",
    "SYBX",
    "SYF",
    "SYK",
    "SYKE",
    "SYMC",
    "SYMX",
    "SYN",
    "SYNA",
    "SYNC",
    "SYNL",
    "SYNT",
    "SYPR",
    "SYRS",
    "SYT",
    "SYX",
    "SYY",
    "SZC",
    "T",
    "TA",
    "TAC",
    "TACO",
    "TACOW",
    "TACT",
    "TAHO",
    "TAIL",
    "TAIT",
    "TAL",
    "TANH",
    "TANNI",
    "TANNL",
    "TANNZ",
    "TAP-A",
    "TAP",
    "TAPR",
    "TARO",
    "TAST",
    "TAT",
    "TATT",
    "TAX",
    "TAXR",
    "TAYD",
    "TBB",
    "TBBK",
    "TBI",
    "TBK",
    "TBLU",
    "TBNK",
    "TBPH",
    "TCAP",
    "TCBI",
    "TCBIL",
    "TCBIP",
    "TCBIW",
    "TCBK",
    "TCCA",
    "TCCB",
    "TCCO",
    "TCF-WS",
    "TCF",
    "TCFC",
    "TCF_B-CL",
    "TCF_C",
    "TCF_D",
    "TCGP",
    "TCHF",
    "TCI",
    "TCMD",
    "TCO",
    "TCON",
    "TCO_J",
    "TCO_K",
    "TCP",
    "TCPC",
    "TCRD",
    "TCRX",
    "TCRZ",
    "TCS",
    "TCTL",
    "TCX",
    "TD",
    "TDA",
    "TDC",
    "TDE",
    "TDF",
    "TDG",
    "TDI",
    "TDJ",
    "TDOC",
    "TDS",
    "TDW-WS-A",
    "TDW-WS-B",
    "TDW",
    "TDY",
    "TEAM",
    "TEAR",
    "TECD",
    "TECH",
    "TECK",
    "TEDU",
    "TEF",
    "TEGP",
    "TEI",
    "TEL",
    "TELL",
    "TEN",
    "TENX",
    "TEO",
    "TEP",
    "TER",
    "TERM",
    "TERP",
    "TESO",
    "TESS",
    "TEST",
    "TEST",
    "TETF",
    "TEUM",
    "TEVA",
    "TEX",
    "TFSL",
    "TFX",
    "TG",
    "TGA",
    "TGB",
    "TGC",
    "TGEN",
    "TGH",
    "TGI",
    "TGLS",
    "TGNA",
    "TGP",
    "TGP_A",
    "TGP_B",
    "TGS",
    "TGT",
    "TGTX",
    "THC",
    "THFF",
    "THG",
    "THGA",
    "THM",
    "THO",
    "THQ",
    "THR",
    "THRM",
    "THS",
    "THST",
    "THW",
    "TI-A",
    "TI",
    "TICC",
    "TICCL",
    "TIER",
    "TIF",
    "TIG",
    "TIK",
    "TIL",
    "TILE",
    "TIME",
    "TIPT",
    "TIS",
    "TISA",
    "TISI",
    "TITN",
    "TIVO",
    "TJX",
    "TK",
    "TKAT",
    "TKC",
    "TKF",
    "TKR",
    "TLDH",
    "TLEH",
    "TLF",
    "TLGT",
    "TLI",
    "TLK",
    "TLND",
    "TLP",
    "TLRA",
    "TLRD",
    "TLYS",
    "TM",
    "TMHC",
    "TMK",
    "TMK_B",
    "TMK_C",
    "TMO",
    "TMP",
    "TMQ",
    "TMST",
    "TMUS",
    "TMUSP",
    "TNAV",
    "TNC",
    "TNDM",
    "TNET",
    "TNH",
    "TNK",
    "TNP",
    "TNP_B",
    "TNP_C",
    "TNP_D",
    "TNP_E",
    "TNTR",
    "TNXP",
    "TOCA",
    "TOL",
    "TOO",
    "TOO_A",
    "TOO_B",
    "TOPS",
    "TORM",
    "TOT",
    "TOUR",
    "TOWN",
    "TOWR",
    "TPB",
    "TPC",
    "TPGE-U",
    "TPGE-WS",
    "TPGE",
    "TPGH-U",
    "TPGH-WS",
    "TPGH",
    "TPH",
    "TPHS",
    "TPIC",
    "TPIV",
    "TPL",
    "TPOR",
    "TPR",
    "TPRE",
    "TPVG",
    "TPVY",
    "TPX",
    "TPYP",
    "TPZ",
    "TR",
    "TRC",
    "TRCB",
    "TRCH",
    "TRCO",
    "TRCR",
    "TRCRW",
    "TREC",
    "TREE",
    "TREX",
    "TRGP",
    "TRHC",
    "TRI",
    "TRIB",
    "TRIL",
    "TRIP",
    "TRK",
    "TRMB",
    "TRMK",
    "TRMT",
    "TRN",
    "TRNC",
    "TRNO",
    "TRNS",
    "TROV",
    "TROW",
    "TROX",
    "TRP",
    "TRPX",
    "TRQ",
    "TRS",
    "TRST",
    "TRT",
    "TRTN",
    "TRTX",
    "TRU",
    "TRUE",
    "TRUP",
    "TRV",
    "TRVG",
    "TRVN",
    "TRX",
    "TRXC",
    "TS",
    "TSBK",
    "TSC",
    "TSCO",
    "TSE",
    "TSEM",
    "TSG",
    "TSI",
    "TSLA",
    "TSLF",
    "TSLX",
    "TSM",
    "TSN",
    "TSQ",
    "TSRI",
    "TSRO",
    "TSS",
    "TST",
    "TSU",
    "TTAC",
    "TTAI",
    "TTC",
    "TTD",
    "TTEC",
    "TTEK",
    "TTF",
    "TTGT",
    "TTI",
    "TTM",
    "TTMI",
    "TTNP",
    "TTOO",
    "TTP",
    "TTPH",
    "TTS",
    "TTWO",
    "TU",
    "TUES",
    "TUP",
    "TURN",
    "TUSK",
    "TV",
    "TVC",
    "TVE",
    "TVPT",
    "TVTY",
    "TWI",
    "TWIN",
    "TWLO",
    "TWMC",
    "TWN",
    "TWNK",
    "TWNKW",
    "TWO",
    "TWOU",
    "TWOW",
    "TWO_A",
    "TWO_B",
    "TWTR",
    "TWX",
    "TX",
    "TXMD",
    "TXN",
    "TXRH",
    "TXT",
    "TY",
    "TYG",
    "TYHT",
    "TYL",
    "TYME",
    "TYPE",
    "TY_",
    "TZOO",
    "UA",
    "UAA",
    "UAL",
    "UAMY",
    "UAN",
    "UBA",
    "UBCP",
    "UBFO",
    "UBIO",
    "UBNK",
    "UBNT",
    "UBOH",
    "UBP",
    "UBP_F-CL",
    "UBP_G",
    "UBP_H",
    "UBRT",
    "UBS",
    "UBSH",
    "UBSI",
    "UCBA",
    "UCBI",
    "UCFC",
    "UCTT",
    "UDBI",
    "UDR",
    "UE",
    "UEC",
    "UEIC",
    "UEPS",
    "UEVM",
    "UFAB",
    "UFCS",
    "UFI",
    "UFPI",
    "UFPT",
    "UFS",
    "UG",
    "UGI",
    "UGP",
    "UHAL",
    "UHS",
    "UHT",
    "UIHC",
    "UIS",
    "UITB",
    "UIVM",
    "UL",
    "ULBI",
    "ULH",
    "ULTA",
    "ULTI",
    "ULVM",
    "UMBF",
    "UMC",
    "UMH",
    "UMH_B",
    "UMH_C",
    "UMPQ",
    "UN",
    "UNAM",
    "UNB",
    "UNF",
    "UNFI",
    "UNH",
    "UNIT",
    "UNM",
    "UNP",
    "UNT",
    "UNTY",
    "UNVR",
    "UONE",
    "UONEK",
    "UPL",
    "UPLD",
    "UPS",
    "UQM",
    "URBN",
    "URG",
    "URGN",
    "URI",
    "USA",
    "USAC",
    "USAK",
    "USAP",
    "USAS",
    "USAT",
    "USATP",
    "USAU",
    "USB",
    "USB_A",
    "USB_H",
    "USB_M",
    "USB_O",
    "USCR",
    "USDP",
    "USEG",
    "USEQ",
    "USFD",
    "USG",
    "USHY",
    "USLB",
    "USLM",
    "USM",
    "USMC",
    "USMF",
    "USNA",
    "USOD",
    "USOI",
    "USOU",
    "USPH",
    "USRT",
    "USTB",
    "USVM",
    "UTES",
    "UTF",
    "UTG",
    "UTHR",
    "UTI",
    "UTL",
    "UTLF",
    "UTMD",
    "UTSI",
    "UTSL",
    "UTX",
    "UUU",
    "UUUU-WS",
    "UUUU",
    "UVE",
    "UVSP",
    "UVV",
    "UWN",
    "UWT",
    "UZA",
    "UZB",
    "UZC",
    "V",
    "VAC",
    "VALE-P",
    "VALE",
    "VALU",
    "VALX",
    "VAMO",
    "VAR",
    "VBF",
    "VBFC",
    "VBIV",
    "VBLT",
    "VBND",
    "VBTX",
    "VC",
    "VCEL",
    "VCF",
    "VCO",
    "VCRA",
    "VCV",
    "VCYT",
    "VDSI",
    "VDTH",
    "VEAC",
    "VEACU",
    "VEACW",
    "VEC",
    "VECO",
    "VEDL",
    "VEEV",
    "VEON",
    "VER",
    "VERI",
    "VERU",
    "VER_F",
    "VESH",
    "VET",
    "VFC",
    "VFL",
    "VG",
    "VGFO",
    "VGI",
    "VGM",
    "VGR",
    "VGZ",
    "VHC",
    "VHI",
    "VIA",
    "VIAB",
    "VIAV",
    "VICL",
    "VICR",
    "VIGI",
    "VII",
    "VIPS",
    "VIRC",
    "VIRT",
    "VISI",
    "VIST",
    "VIV",
    "VIVE",
    "VIVO",
    "VJET",
    "VKI",
    "VKQ",
    "VKTX",
    "VKTXW",
    "VLGEA",
    "VLO",
    "VLP",
    "VLRS",
    "VLRX",
    "VLT",
    "VLY-WS",
    "VLY",
    "VLY_A",
    "VLY_B",
    "VMAX",
    "VMC",
    "VMET",
    "VMI",
    "VMIN",
    "VMM",
    "VMO",
    "VMOT",
    "VMW",
    "VNCE",
    "VNDA",
    "VNET",
    "VNLA",
    "VNO",
    "VNOM",
    "VNO_G",
    "VNO_I",
    "VNO_K",
    "VNO_L",
    "VNRX",
    "VNTR",
    "VNTV",
    "VOC",
    "VOD",
    "VOXX",
    "VOYA",
    "VPG",
    "VPV",
    "VR",
    "VRA",
    "VRAY",
    "VREX",
    "VRIG",
    "VRML",
    "VRNA",
    "VRNS",
    "VRNT",
    "VRS",
    "VRSK",
    "VRSN",
    "VRTS",
    "VRTSP",
    "VRTU",
    "VRTV",
    "VRTX",
    "VRX",
    "VR_A",
    "VR_B",
    "VSAR",
    "VSAT",
    "VSDA",
    "VSEC",
    "VSH",
    "VSI",
    "VSLR",
    "VSM",
    "VSMV",
    "VST",
    "VSTM",
    "VSTO",
    "VTA",
    "VTC",
    "VTEB",
    "VTGN",
    "VTL",
    "VTN",
    "VTNR",
    "VTR",
    "VTRB",
    "VTVT",
    "VUZI",
    "VVC",
    "VVI",
    "VVPR",
    "VVR",
    "VVUS",
    "VVV",
    "VWR",
    "VYGR",
    "VYMI",
    "VZ",
    "VZA",
    "W",
    "WAAS",
    "WAB",
    "WABC",
    "WAC",
    "WAFD",
    "WAFDW",
    "WAGE",
    "WAIR",
    "WAL",
    "WALA",
    "WASH",
    "WAT",
    "WATT",
    "WAYN",
    "WB",
    "WBA",
    "WBAI",
    "WBBW",
    "WBC",
    "WBIA",
    "WBIB",
    "WBIC",
    "WBID",
    "WBIE",
    "WBIF",
    "WBIG",
    "WBIH",
    "WBII",
    "WBIL",
    "WBIR",
    "WBIY",
    "WBK",
    "WBKC",
    "WBS",
    "WBS_E",
    "WBT",
    "WCC",
    "WCFB",
    "WCG",
    "WCN",
    "WD",
    "WDAY",
    "WDC",
    "WDFC",
    "WDR",
    "WDRW",
    "WEA",
    "WEAR",
    "WEB",
    "WEBK",
    "WEC",
    "WEN",
    "WERN",
    "WES",
    "WETF",
    "WEX",
    "WEXP",
    "WEYS",
    "WF",
    "WFBI",
    "WFC-WS",
    "WFC",
    "WFC_J",
    "WFC_L",
    "WFC_N",
    "WFC_O",
    "WFC_P",
    "WFC_Q",
    "WFC_R",
    "WFC_T",
    "WFC_V",
    "WFC_W",
    "WFC_X",
    "WFC_Y",
    "WFE_A",
    "WFHY",
    "WFIG",
    "WFT",
    "WG",
    "WGL",
    "WGO",
    "WGP",
    "WHF",
    "WHFBL",
    "WHG",
    "WHLM",
    "WHLR",
    "WHLRD",
    "WHLRP",
    "WHLRW",
    "WHR",
    "WIA",
    "WIFI",
    "WILC",
    "WIN",
    "WINA",
    "WING",
    "WINS",
    "WIRE",
    "WIT",
    "WIW",
    "WIX",
    "WK",
    "WKHS",
    "WLB",
    "WLDN",
    "WLFC",
    "WLH",
    "WLK",
    "WLKP",
    "WLL",
    "WLTW",
    "WM",
    "WMB",
    "WMC",
    "WMGI",
    "WMGIZ",
    "WMIH",
    "WMK",
    "WMLP",
    "WMS",
    "WMT",
    "WNC",
    "WNEB",
    "WNFM",
    "WNRL",
    "WNS",
    "WOR",
    "WOW",
    "WPC",
    "WPCS",
    "WPG",
    "WPG_H",
    "WPG_I",
    "WPM",
    "WPPGY",
    "WPRT",
    "WPX",
    "WPXP",
    "WPZ",
    "WR",
    "WRB",
    "WRB_B",
    "WRB_C",
    "WRB_D",
    "WRD",
    "WRE",
    "WRI",
    "WRK",
    "WRLD",
    "WRLS",
    "WRLSR",
    "WRLSU",
    "WRLSW",
    "WRN",
    "WSBC",
    "WSBF",
    "WSCI",
    "WSFS",
    "WSKY",
    "WSM",
    "WSO-B",
    "WSO",
    "WSPT",
    "WSR",
    "WST",
    "WSTG",
    "WSTL",
    "WTBA",
    "WTFC",
    "WTFCM",
    "WTFCW",
    "WTI",
    "WTID",
    "WTIU",
    "WTM",
    "WTR",
    "WTRX",
    "WTS",
    "WTT",
    "WTTR",
    "WTW",
    "WU",
    "WUBA",
    "WUSA",
    "WVE",
    "WVFC",
    "WVVI",
    "WVVIP",
    "WWD",
    "WWE",
    "WWR",
    "WWW",
    "WY",
    "WYDE",
    "WYIG",
    "WYIGU",
    "WYIGW",
    "WYN",
    "WYNN",
    "WYY",
    "X",
    "XBIO",
    "XBIT",
    "XBKS",
    "XCEM",
    "XCO",
    "XCRA",
    "XEC",
    "XEL",
    "XELA",
    "XELB",
    "XENE",
    "XENT",
    "XFLT",
    "XGTI",
    "XGTIW",
    "XHR",
    "XIN",
    "XINA",
    "XITK",
    "XIVH",
    "XL",
    "XLNX",
    "XLRE",
    "XLRN",
    "XMX",
    "XNCR",
    "XNET",
    "XNTK",
    "XNY",
    "XOG",
    "XOM",
    "XOMA",
    "XON",
    "XONE",
    "XOXO",
    "XPER",
    "XPL",
    "XPLR",
    "XPO",
    "XRAY",
    "XRF",
    "XRM",
    "XRX",
    "XSHD",
    "XSHQ",
    "XTH",
    "XTLB",
    "XTNT",
    "XUSA",
    "XWEB",
    "XXII",
    "XYL",
    "Y",
    "YECO",
    "YELP",
    "YERR",
    "YESR",
    "YEXT",
    "YGE",
    "YGYI",
    "YIN",
    "YLD",
    "YLDE",
    "YNDX",
    "YOGA",
    "YORW",
    "YPF",
    "YRCW",
    "YRD",
    "YTEN",
    "YTRA",
    "YUM",
    "YUMA",
    "YUMC",
    "YUME",
    "YY",
    "Z",
    "ZAGG",
    "ZAIS",
    "ZAYO",
    "ZBH",
    "ZBIO",
    "ZBK",
    "ZBRA",
    "ZBZX",
    "ZB_A",
    "ZB_G",
    "ZB_H",
    "ZDGE",
    "ZEAL",
    "ZEN",
    "ZEUS",
    "ZF",
    "ZFGN",
    "ZG",
    "ZGNX",
    "ZION",
    "ZIONW",
    "ZIONZ",
    "ZIOP",
    "ZIXI",
    "ZJZZT",
    "ZKIN",
    "ZLAB",
    "ZN",
    "ZNGA",
    "ZNH",
    "ZNWAA",
    "ZOES",
    "ZSAN",
    "ZTO",
    "ZTR",
    "ZTS",
    "ZUMZ",
    "ZVV",
    "ZX",
    "ZYME",
    "ZYNE",
] """
""" 
etfList = ['AADR', 'AAXJ', 'ACIM', 'ACTX', 'ACWF', 'ACWI', 'ACWV', 'ACWX', 'ADRA', 'ADRD', 'ADRE', 'ADRU', 'AFK', 'AGF', 'AGG', 'AGND', 'AGQ', 'AGZ', 'AGZD', 'AIA', 'AIRR', 'ALD', 'ALFA', 'ALTS', 'AMJ', 'AMJL', 'AMLP', 'AMU', 'ANGL', 'AOA', 'AOK', 'AOM', 'AOR', 'ARGT', 'ASEA', 'ASHR', 'ASHS', 'ATMP', 'AUSE', 'AXJL', 'AXJV', 'AYT', 'BAB', 'BAL', 'BBH', 'BBP', 'BBRC', 'BCM', 'BDCL', 'BDCS', 'BDD', 'BFOR', 'BIB', 'BICK', 'BIL', 'BIS', 'BIV', 'BIZD', 'BJK', 'BKF', 'BKLN', 'BLV', 'BND', 'BNDX', 'BNO', 'BOIL', 'BOM', 'BOND', 'BRAQ', 'BRF', 'BRZU', 'BSCH', 'BSCI', 'BSCJ', 'BSCK', 'BSCL', 'BSCM', 'BSCN', 'BSCO', 'BSJH', 'BSJI', 'BSJJ', 'BSJK', 'BSJL', 'BSJM', 'BSV', 'BTAL', 'BWV', 'BWX', 'BWZ', 'BYLD', 'BZF', 'BZQ', 'CAFE', 'CANE', 'CAPE', 'CARZ', 'CBND', 'CBON', 'CEFL', 'CEW', 'CGW', 'CHAU', 'CHEP', 'CHIE', 'CHII', 'CHIM', 'CHIQ', 'CHIX', 'CHOC', 'CIU', 'CJNK', 'CLY', 'CMBS', 'CMDT', 'CMF', 'CN', 'CNDA', 'CNTR', 'CNXT', 'CNY', 'COMT', 'COPX', 'CORN', 'CORP', 'COW', 'CPER', 'CPI', 'CQQQ', 'CRBN', 'CRED', 'CROC', 'CROP', 'CSD', 'CSJ', 'CSM', 'CTNN', 'CUPM', 'CURE', 'CUT', 'CVY', 'CWB', 'CWI', 'CYB', 'CZA', 'DAG', 'DBA', 'DBAP', 'DBAW', 'DBB', 'DBBR', 'DBC', 'DBE', 'DBEF', 'DBEM', 'DBEU', 'DBEZ', 'DBGR', 'DBJP', 'DBKO', 'DBMX', 'DBO', 'DBP', 'DBS', 'DBUK', 'DBV', 'DCNG', 'DDG', 'DDM', 'DDP', 'DEF', 'DEM', 'DES', 'DEW', 'DFE', 'DFJ', 'DGAZ', 'DGL', 'DGLD', 'DGP', 'DGRE', 'DGRO', 'DGRS', 'DGRW', 'DGS', 'DGT', 'DGZ', 'DHS', 'DIA', 'DIG', 'DIM', 'DIRT', 'DIV', 'DJCI', 'DJP', 'DLN', 'DLS', 'DNL', 'DNO', 'DOD', 'DOG', 'DOL', 'DON', 'DOO', 'DPK', 'DRIP', 'DRN', 'DRR', 'DRV', 'DRW', 'DSI', 'DSLV', 'DSUM', 'DTD', 'DTH', 'DTN', 'DTO', 'DUG', 'DUST', 'DVHL', 'DVY', 'DVYA', 'DVYE', 'DVYL', 'DWM', 'DWX', 'DXD', 'DXGE', 'DXJ', 'DXJC', 'DXJF', 'DXJH', 'DXJR', 'DXJS', 'DXPS', 'DYY', 'DZK', 'DZZ', 'EBND', 'ECH', 'ECNS', 'ECON', 'EDC', 'EDEN', 'EDIV', 'EDOG', 'EDV', 'EDZ', 'EEB', 'EEH', 'EELV', 'EEM', 'EEMA', 'EEMS', 'EEMV', 'EES', 'EET', 'EEV', 'EFA', 'EFAD', 'EFAV', 'EFFE', 'EFG', 'EFO', 'EFU', 'EFV', 'EFZ', 'EGPT', 'EIDO', 'EIRL', 'EIS', 'ELD', 'EMAG', 'EMB', 'EMCB', 'EMCG', 'EMFM', 'EMIF', 'EMIH', 'EMLC', 'EMLP', 'EMSH', 'ENFR', 'ENY', 'ENZL', 'EPHE', 'EPI', 'EPOL', 'EPP', 'EPS', 'EPU', 'EPV', 'EQAL', 'EQL', 'EQLT', 'EQWL', 'EQWM', 'EQWS', 'ERO', 'ERUS', 'ERX', 'ERY', 'EUDG', 'EUFN', 'EUFX', 'EUM', 'EUMV', 'EUO', 'EURL', 'EUSA', 'EUSC', 'EVX', 'EWA', 'EWC', 'EWD', 'EWEM', 'EWG', 'EWH', 'EWI', 'EWJ', 'EWK', 'EWL', 'EWM', 'EWN', 'EWO', 'EWP', 'EWQ', 'EWS', 'EWT', 'EWU', 'EWV', 'EWW', 'EWX', 'EWY', 'EWZ', 'EWZS', 'EXI', 'EXT', 'EZA', 'EZJ', 'EZM', 'EZU', 'EZY', 'FAB', 'FAD', 'FAM', 'FAN', 'FAS', 'FAUS', 'FAZ', 'FBGX', 'FBND', 'FBT', 'FBZ', 'FCA', 'FCAN', 'FCG', 'FCOM', 'FCOR', 'FCT', 'FDD', 'FDIS', 'FDIV', 'FDL', 'FDM', 'FDN', 'FDT', 'FDTS', 'FEEU', 'FEI', 'FEM', 'FEMB', 'FEMS', 'FENY', 'FEO', 'FEP', 'FEU', 'FEUZ', 'FEX', 'FEZ', 'FFA', 'FFR', 'FGB', 'FGD', 'FGM', 'FHK', 'FHLC', 'FHY', 'FIDU', 'FIEG', 'FIEU', 'FIF', 'FIGY', 'FILL', 'FINU', 'FINZ', 'FIW', 'FJP', 'FKO', 'FKU', 'FLAG', 'FLGE', 'FLM', 'FLN', 'FLOT', 'FLRN', 'FLRT', 'FLTB', 'FLTR', 'FM', 'FMAT', 'FMB', 'FMF', 'FMK', 'FMY', 'FNCL', 'FNDA', 'FNDB', 'FNDC', 'FNDE', 'FNDF', 'FNDX', 'FNI', 'FNK', 'FNX', 'FNY', 'FOIL', 'FONE', 'FPA', 'FPE', 'FPF', 'FPL', 'FPX', 'FPXI', 'FRAK', 'FREL', 'FRI', 'FRN', 'FSD', 'FSTA', 'FSZ', 'FTA', 'FTC', 'FTCS', 'FTEC', 'FTGC', 'FTHI', 'FTLB', 'FTLS', 'FTSD', 'FTSL', 'FTSM', 'FTW', 'FUD', 'FUE', 'FUTY', 'FV', 'FVD', 'FVL', 'FWDB', 'FWDD', 'FWDI', 'FXA', 'FXB', 'FXC', 'FXCH', 'FXD', 'FXE', 'FXEU', 'FXF', 'FXG', 'FXH', 'FXI', 'FXL', 'FXN', 'FXO', 'FXP', 'FXR', 'FXS', 'FXSG', 'FXU', 'FXY', 'FXZ', 'FYC', 'FYLD', 'FYT', 'FYX', 'GAA', 'GAL', 'GASL', 'GAZ', 'GBB', 'GBF', 'GCC', 'GCE', 'GDX', 'GDXJ', 'GDXS', 'GDXX', 'GEUR', 'GEX', 'GHII', 'GII', 'GLD', 'GLDI', 'GLL', 'GLTR', 'GMF', 'GMOM', 'GNMA', 'GNR', 'GOVT', 'GQRE', 'GREK', 'GRES', 'GRI', 'GRID', 'GRN', 'GRU', 'GRWN', 'GSC', 'GSG', 'GSP', 'GSY', 'GULF', 'GUNR', 'GURU', 'GUSH', 'GVAL', 'GVI', 'GWX', 'GXC', 'GXF', 'GXG', 'GYEN', 'GYLD', 'HAO', 'HAP', 'HDG', 'HDGE', 'HDV', 'HECO', 'HEDJ', 'HEEM', 'HEFA', 'HEVY', 'HEWG', 'HEWJ', 'HEZU', 'HGI', 'HILO', 'HOLD', 'HSPX', 'HUSE', 'HYD', 'HYEM', 'HYG', 'HYGH', 'HYIH', 'HYLD', 'HYLS', 'HYMB', 'HYND', 'HYS', 'HYZD', 'IAI', 'IAK', 'IAT', 'IAU', 'IBB', 'IBCC', 'IBCD', 'IBCE', 'IBDB', 'IBDC', 'IBDD', 'IBDH', 'IBDJ', 'IBDK', 'IBDL', 'IBDM', 'IBDN', 'IBDO', 'IBDP', 'IBDQ', 'IBLN', 'IBMG', 'IBMH', 'IBMI', 'IBND', 'ICF', 'ICI', 'ICLN', 'ICOL', 'IDHQ', 'IDLV', 'IDOG', 'IDU', 'IDV', 'IDX', 'IEF', 'IEFA', 'IEI', 'IEMG', 'IEO', 'IEUR', 'IEUS', 'IEV', 'IEZ', 'IFEU', 'IFGL', 'IFV', 'IGE', 'IGF', 'IGIH', 'IGM', 'IGN', 'IGOV', 'IGV', 'IHDG', 'IHE', 'IHF', 'IHI', 'IHY', 'IJH', 'IJJ', 'IJK', 'IJR', 'IJS', 'IJT', 'ILF', 'ILTB', 'IMLP', 'IMTM', 'INCO', 'INDA', 'INDL', 'INDY', 'INKM', 'INP', 'INR', 'INTF', 'INXX', 'IOIL', 'IOO', 'IPAC', 'IPE', 'IPFF', 'IPKW', 'IPO', 'IQDE', 'IQDF', 'IQDY', 'IQLT', 'ISCF', 'ISHG', 'ISRA', 'ISTB', 'ITA', 'ITB', 'ITE', 'ITM', 'ITOT', 'IUSB', 'IUSG', 'IUSV', 'IVE', 'IVOG', 'IVOO', 'IVOV', 'IVV', 'IVW', 'IWB', 'IWC', 'IWD', 'IWF', 'IWL', 'IWM', 'IWN', 'IWO', 'IWP', 'IWR', 'IWS', 'IWV', 'IWX', 'IWY', 'IXC', 'IXG', 'IXJ', 'IXN', 'IXP', 'IXUS', 'IYC', 'IYE', 'IYF', 'IYG', 'IYH', 'IYJ', 'IYK', 'IYM', 'IYR', 'IYT', 'IYW', 'IYY', 'IYZ', 'JDST', 'JHDG', 'JJA', 'JJC', 'JJE', 'JJG', 'JJM', 'JJN', 'JJP', 'JJS', 'JJT', 'JJU', 'JKD', 'JKE', 'JKF', 'JKG', 'JKH', 'JKI', 'JKJ', 'JKK', 'JKL', 'JNK', 'JNUG', 'JO', 'JPGE', 'JPMV', 'JPNL', 'JXI', 'KBA', 'KBE', 'KBWP', 'KBWR', 'KCE', 'KFYP', 'KIE', 'KNOW', 'KOL', 'KOLD', 'KORU', 'KRE', 'KROO', 'KXI', 'LABD', 'LABU', 'LALT', 'LBJ', 'LDRI', 'LDUR', 'LEMB', 'LGLV', 'LIT', 'LLSC', 'LLSP', 'LMBS', 'LMLP', 'LOWC', 'LQD', 'LQDH', 'LRGF', 'LSTK', 'LTL', 'LTPZ', 'LVL', 'MATH', 'MBB', 'MBG', 'MBSD', 'MCHI', 'MCRO', 'MDIV', 'MDY', 'MDYG', 'MDYV', 'MFD', 'MGC', 'MGK', 'MGV', 'MIDU', 'MIDZ', 'MINC', 'MINT', 'MLN', 'MLPA', 'MLPC', 'MLPG', 'MLPI', 'MLPS', 'MLPX', 'MLPY', 'MMTM', 'MNA', 'MOAT', 'MOM', 'MOO', 'MORL', 'MORT', 'MRGR', 'MTUM', 'MUB', 'MUNI', 'MVV', 'MXI', 'MYY', 'MZZ', 'NASH', 'NEAR', 'NFO', 'NFRA', 'NGE', 'NIB', 'NINI', 'NLR', 'NOBL', 'NORW', 'NUGT', 'NYF', 'OEF', 'OIH', 'OIL', 'OLEM', 'OLO', 'ONEQ', 'OUNZ', 'PAF', 'PAGG', 'PAK', 'PALL', 'PBD', 'PBE', 'PBJ', 'PBP', 'PBS', 'PBW', 'PCEF', 'PCY', 'PDBC', 'PDN', 'PEJ', 'PEK', 'PEX', 'PEY', 'PEZ', 'PFF', 'PFIG', 'PFM', 'PFXF', 'PGAL', 'PGF', 'PGHY', 'PGJ', 'PGM', 'PGX', 'PHB', 'PHDG', 'PICB', 'PICK', 'PIE', 'PIN', 'PIO', 'PJP', 'PKB', 'PLND', 'PMR', 'PNQI', 'PPA', 'PPH', 'PPLT', 'PRB', 'PRF', 'PRFZ', 'PSAU', 'PSCC', 'PSCD', 'PSCE', 'PSCF', 'PSCH', 'PSCI', 'PSCM', 'PSCT', 'PSCU', 'PSI', 'PSJ', 'PSK', 'PSP', 'PSQ', 'PSR', 'PST', 'PTM', 'PUW', 'PVI', 'PWB', 'PWC', 'PWV', 'PWZ', 'PXE', 'PXF', 'PXH', 'PXJ', 'PXLG', 'PXLV', 'PXMG', 'PXMV', 'PXQ', 'PXR', 'PXSG', 'PXSV', 'PZA', 'PZD', 'PZI', 'PZT', 'QABA', 'QAI', 'QAT', 'QCAN', 'QCLN', 'QDEF', 'QDEU', 'QDF', 'QDYN', 'QED', 'QEFA', 'QEMM', 'QGBR', 'QID', 'QINC', 'QJPN', 'QLD', 'QLS', 'QLTA', 'QMN', 'QQEW', 'QQQ', 'QQQC', 'QQQE', 'QQXT', 'QTEC', 'QUAL', 'QUS', 'QWLD', 'QYLD', 'RALS', 'RAVI', 'RCD', 'RDIV', 'RDVY', 'REET', 'REGL', 'REK', 'REM', 'REMX', 'RETL', 'REW', 'REZ', 'RFG', 'RFV', 'RGI', 'RHS', 'RIGS', 'RINF', 'RING', 'RJA', 'RJI', 'RJN', 'RJZ', 'RLY', 'ROBO', 'ROLA', 'ROM', 'ROOF', 'RPG', 'RPV', 'RSP', 'RSX', 'RSXJ', 'RTH', 'RTM', 'RUSL', 'RUSS', 'RVNU', 'RWJ', 'RWK', 'RWL', 'RWM', 'RWO', 'RWR', 'RWW', 'RWX', 'RXD', 'RXI', 'RXL', 'RYE', 'RYF', 'RYH', 'RYJ', 'RYT', 'RYU', 'RZG', 'RZV', 'SAA', 'SAGG', 'SBB', 'SBIO', 'SBM', 'SCC', 'SCHA', 'SCHB', 'SCHC', 'SCHD', 'SCHE', 'SCHF', 'SCHG', 'SCHH', 'SCHM', 'SCHO', 'SCHP', 'SCHR', 'SCHV', 'SCHX', 'SCHZ', 'SCID', 'SCIF', 'SCIJ', 'SCIN', 'SCIU', 'SCIX', 'SCJ', 'SCO', 'SCTO', 'SCZ', 'SDD', 'SDEM', 'SDIV', 'SDOG', 'SDOW', 'SDP', 'SDS', 'SDY', 'SDYL', 'SEA', 'SEF', 'SGAR', 'SGDJ', 'SGDM', 'SGG', 'SGOL', 'SH', 'SHM', 'SHV', 'SHY', 'SHYD', 'SHYG', 'SIJ', 'SIL', 'SILJ', 'SIVR', 'SIZ', 'SIZE', 'SJB', 'SJNK', 'SKF', 'SKOR', 'SKYY', 'SLQD', 'SLV', 'SLVO', 'SLVP', 'SLX', 'SLY', 'SLYG', 'SLYV', 'SMB', 'SMDD', 'SMDV', 'SMEZ', 'SMH', 'SMLF', 'SMLL', 'SMLV', 'SMMU', 'SMN', 'SNLN', 'SOCL', 'SOIL', 'SOXL', 'SOXS', 'SOXX', 'SOYB', 'SPFF', 'SPHB', 'SPHD', 'SPHQ', 'SPLV', 'SPLX', 'SPUU', 'SPXH', 'SPXL', 'SPXS', 'SPXU', 'SPY', 'SPYB', 'SPYG', 'SPYV', 'SQQQ', 'SRET', 'SRLN', 'SRS', 'SRTY', 'SSG', 'SSO', 'STIP', 'STPZ', 'SUB', 'SVXY', 'SYE', 'SYG', 'SYLD', 'SYV', 'SZK', 'TAGS', 'TAN', 'TAO', 'TBF', 'TBT', 'TBX', 'TDIV', 'TDTF', 'TDTT', 'TECL', 'TECS', 'TEST', 'TFI', 'TFLO', 'THD', 'TILT', 'TIP', 'TIPX', 'TIPZ', 'TLH', 'TLT', 'TLTD', 'TLTE', 'TMF', 'TMV', 'TNA', 'TOK', 'TOLZ', 'TOTL', 'TQQQ', 'TRSK', 'TTFS', 'TTT', 'TUR', 'TUSA', 'TUZ', 'TVIX', 'TVIZ', 'TWM', 'TYBS', 'TYD', 'TYNS', 'TYO', 'TZA', 'UAE', 'UAG', 'UBC', 'UBG', 'UBM', 'UBN', 'UBR', 'UBT', 'UCC', 'UCI', 'UCO', 'UDN', 'UDOW', 'UGA', 'UGAZ', 'UGE', 'UGL', 'UGLD', 'UHN', 'UJB', 'ULE', 'ULST', 'UMDD', 'UNG', 'UNL', 'UPRO', 'UPV', 'UPW', 'URA', 'URE', 'URR', 'URTH', 'URTY', 'USAG', 'USCI', 'USD', 'USDU', 'USFR', 'USL', 'USLV', 'USMV', 'USO', 'UST', 'USV', 'UUP', 'UVXY', 'UWM', 'UXI', 'UYG', 'UYM', 'VAW', 'VB', 'VBK', 'VBR', 'VCIT', 'VCLT', 'VCR', 'VCSH', 'VDC', 'VDE', 'VEA', 'VEGA', 'VEGI', 'VEU', 'VFH', 'VGIT', 'VGK', 'VGLT', 'VGSH', 'VGT', 'VHT', 'VIDI', 'VIG', 'VIIX', 'VIIZ', 'VIOG', 'VIOO', 'VIOV', 'VIS', 'VIXM', 'VIXY', 'VLU', 'VLUE', 'VMBS', 'VNM', 'VNQ', 'VNQI', 'VO', 'VOE', 'VONE', 'VONG', 'VONV', 'VOO', 'VOOG', 'VOOV', 'VOT', 'VOX', 'VPL', 'VPU', 'VQT', 'VRP', 'VSS', 'VT', 'VTHR', 'VTI', 'VTIP', 'VTV', 'VTWG', 'VTWO', 'VTWV', 'VUG', 'VUSE', 'VV', 'VWO', 'VWOB', 'VXF', 'VXUS', 'VXX', 'VXZ', 'VYM', 'WDIV', 'WDTI', 'WEAT', 'WEET', 'WIP', 'WMCR', 'WMW', 'WOOD', 'WPS', 'WREI', 'XAR', 'XBI', 'XES', 'XHB', 'XHE', 'XHS', 'XIV', 'XLB', 'XLE', 'XLF', 'XLG', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY', 'XME', 'XMLV', 'XMPT', 'XOP', 'XPH', 'XPP', 'XRLV', 'XRT', 'XSD', 'XSLV', 'XSOE', 'XSW', 'XT', 'XTL', 'XTN', 'XVZ', 'YANG', 'YAO', 'YCL', 'YCS', 'YDIV', 'YINN', 'YLCO', 'YMLI', 'YMLP', 'YXI', 'YYY', 'ZIV', 'ZMLP', 'ZROZ', 'ZSL']
 """

stock_list = [
    "TSLA",
    "AMD",
    "GRAB",
    "NVDA",
    "INTC",
    "AAPL",
    "AAL",
    "SOFI",
    "RIVN",
    "MU",
    "AMZN",
    "MSFT",
    "CSX",
]
trainStartDate = "2024-01-01"

for i in stock_list:
    makeModel(i, trainStartDate)
